# PART 4: Current Data Retrieval/ETL

## Get Data for last 5 days

In [1]:
# imports
import pandas as pd
import requests

In [2]:
# simulate login request to fantasydata
frames = []
session_requests = requests.session()

headers = {
    'authority': 'fantasydata.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'origin': 'https://fantasydata.com',
    'referer': 'https://fantasydata.com/user/login',
}

data = {
    'Email': null,
    'Password': null,
    'RedirectUri': '',
    'secondary_email': '',
}

response = session_requests.post('https://fantasydata.com/user/login', headers=headers, data=data)


url = 'https://fantasydata.com/NHL_FantasyStats/FantasyStats_Read'
headers = {
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-encoding': 'gzip, deflate'
    }

# get data
positions = {'5' : 'G', '1' : 'C', '6' : 'D', '2' : 'LW', '3' : 'RW', '4' : 'C_W'}
seasons = list(range(2022, 2023, 1))

from datetime import date, timedelta

# get data
frames = []
for season in seasons:
    sdate = date.today()-timedelta(5)   # start date
    edate = date.today()   # end date
    dates = pd.date_range(sdate,edate-timedelta(days=1),freq='d').tolist()
    for i in range(len(positions)):
        for dateobj in dates:
            month = str(dateobj.month)
            day = str(dateobj.day)
            year = str(dateobj.year)
            if len(month) == 1:
                month = '0' + month
            if len(day) == 1:
                day = '0' + day
            myTuple = month, day, year
            datestr = ('-').join(myTuple)
            formdata = {
                        'sort': 'FantasyPoints-desc',
                        'pageSize': '300',
                        'filters.position': list(positions.keys())[i],
                        'filters.season': str(season),
                        'filters.seasontype': '1',
                        'filters.scope': '2',
                        'filters.seasontype' : '1',
                        'filters.date' : datestr,
                        'filters.conference': 1
                        }
            resp = session_requests.get(url, headers = headers, data = formdata) #Mimic Web Browser Request To Gather Data
            
            data = resp.json() #Grab Data In JSON format
            df = pd.json_normalize(data, record_path=['Data']) #Transform Json to dataframe
            if len(df) > 0:
                df['Date'] = dateobj
                frames.append(df)
                print('Got data for', list(positions.values())[i], 'on', datestr)
df_all = pd.concat(frames, axis=0)
df_all.reset_index(drop=True, inplace=True)
df_all.to_csv('player_stats_recent.csv')

Got data for G on 02-01-2023
Got data for G on 02-04-2023
Got data for C on 02-01-2023
Got data for C on 02-04-2023
Got data for D on 02-01-2023
Got data for D on 02-04-2023
Got data for LW on 02-01-2023
Got data for LW on 02-04-2023
Got data for RW on 02-01-2023
Got data for RW on 02-04-2023
Got data for C_W on 02-01-2023
Got data for C_W on 02-04-2023


## Merge recent data with data for whole season

In [3]:
# read in dataframes with year to date stats and stats retrieved in previous step
YTD_df = pd.read_csv('player_stats_YTD.csv', index_col = 0)
recent_df = pd.read_csv('player_stats_recent.csv', index_col = 0)

#combine YTD and recent dataframes
frames = [YTD_df, recent_df]
df_all = pd.concat(frames, axis=0)

# drop overlapping entries
df_all = df_all.drop_duplicates(subset = ['Name', 'Date'])

In [4]:
# write to csv
df_all.to_csv('player_stats_YTD.csv')
df_all.to_csv('player_stats_current.csv')

## Current Data ETL

In [5]:
#Load Our Modules & read in data set
import os 
import pandas as pd
import numpy as np
import datetime as dt

working_directory = 'D:/machine_learning/DFS/NHL'
os.chdir(working_directory)
data_dir = 'CurrentData/' #Where is your data located?
etl_dir = 'CurrentData/ETL/' #Where is your output data going?
player_stats = pd.read_csv('player_stats_current.csv', index_col = 0) #Read In Our Main Dataset

# convert date from object dtype to datetime
player_stats['Date'] = pd.to_datetime(player_stats['Date'])

# get month and year from datetime type date
player_stats['Month'], player_stats['Year'] = player_stats['Date'].dt.month, player_stats['Date'].dt.year

# convert date from datetime to string
player_stats['Date'] = player_stats['Date'].dt.strftime('%Y%m%d')

# create a new column and use np.select to assign values to it using our lists as arguments
player_stats['Season'] = 2022

# display updated DataFrame
display(player_stats)

,StatID,TeamID,PlayerID,Name,Team,Position,Games,Started,Season,Goals,...,Game.Quarter,Game.Status,Game.IsOver,Game.TimeRemainingMinutes,Game.TimeRemainingSeconds,Game.DateTime,Game.TimeRemaining,Game.QuarterDisplay,Month,Year
0,2650647,16,30002616,Elvis Merzlikins,CBJ,G,1,0,2022,0.0,...,NaN,Final,True,NaN,NaN,/Date(1664665200000)/,NaN,F,10,2022
1,2650135,2,30003673,Ukko-Pekka Luukkonen,BUF,G,1,0,2022,0.0,...,NaN,Final,True,NaN,NaN,/Date(1664643600000)/,NaN,F,10,2022
2,2650225,8,30004494,Spencer Knight,FLA,G,1,0,2022,0.0,...,NaN,Final,True,NaN,NaN,/Date(1664643600000)/,NaN,F,10,2022
3,2650725,25,30003729,Stuart Skinner,EDM,G,1,0,2022,0.0,...,NaN,F/SO,True,NaN,NaN,/Date(1664668800000)/,NaN,F/SO,10,2022
4,2650080,14,30000392,Tristan Jarry,PIT,G,1,0,2022,0.0,...,NaN,Final,True,NaN,NaN,/Date(1664643600000)/,NaN,F,10,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2762493,2,30004867,JJ Peterka,BUF,RW,1,0,2022,0.0,...,NaN,Final,True,NaN,NaN,/Date(1675297800000)/,NaN,F,2,2023
107,2764512,20,30000797,Vladimir Tarasenko,STL,RW,1,0,2022,1.0,...,NaN,Final,True,NaN,NaN,/Date(1675540800000)/,NaN,F,2,2023
108,2764507,29,30003141,Clayton Keller,ARI,RW,1,0,2022,2.0,...,NaN,Final,True,NaN,NaN,/Date(1675540800000)/,NaN,F,2,2023
109,2764508,19,30000906,Mikko Rantanen,COL,RW,1,1,2022,0.0,...,NaN,Final,True,NaN,NaN,/Date(1675540800000)/,NaN,F,2,2023


In [6]:
""" C ETL """
c_yr_dfs = []
for yr in range(2022, 2023): 
    print(yr)
    # filter to current year and C position
    yr_c = player_stats[(player_stats['Season']==yr) & (player_stats['Position']=='C')].copy().reset_index(drop=True)
    yr_c = yr_c.drop_duplicates(subset = ['Name', 'Date'])
    yr_c.drop(['GoaltendingWins', 'GoaltendingLosses', 'GoaltendingOvertimeLosses', 'GoaltendingShotsAgainst', 'GoaltendingGoalsAgainst', 'GoaltendingSaves', 'GoaltendingShutouts',  'GoaltendingGoalsAgainstAverage', 'GoaltendingSavePercentage', 'GoaltendingMinutes'], axis = 1, inplace = True)

    # sort by players & week
    yr_c.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

    # aggregate stats for last 3 games (sums and means)
    # games
    yr_c['GM3'] = yr_c.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['GM_pg3'] = yr_c.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goals
    yr_c['G3'] = yr_c.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['G_pg3'] = yr_c.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # assists
    yr_c['A3'] = yr_c.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['A_pg3'] = yr_c.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # points
    yr_c['PTS3'] = yr_c.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['PTS_pg3'] = yr_c.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # plusminus
    yr_c['plusminus3'] = yr_c.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['plusminus_pg3'] = yr_c.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Hat Tricks
    yr_c['HT3'] = yr_c.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['HT_pg3'] = yr_c.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Penalty Minutes
    yr_c['PIM3'] = yr_c.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['PIM_pg3'] = yr_c.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # PPG
    yr_c['PPG3'] = yr_c.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['PPG_pg3'] = yr_c.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SHG
    yr_c['SHG3'] = yr_c.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['SHG_pg3'] = yr_c.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SOG
    yr_c['SOG3'] = yr_c.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['SOG_pg3'] = yr_c.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # BLocks
    yr_c['BLK3'] = yr_c.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['BLK_pg3'] = yr_c.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_c['FP3'] = yr_c.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['FP_pg3'] = yr_c.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    
    # aggregate stats for season
    # games
    yr_c['GM'] = yr_c.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    # goals
    yr_c['G'] = yr_c.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['G_pg'] = yr_c.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # assists
    yr_c['A'] = yr_c.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['A_pg'] = yr_c.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # points
    yr_c['PTS'] = yr_c.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['PTS_pg'] = yr_c.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # plusminus
    yr_c['PM'] = yr_c.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['PM_pg'] = yr_c.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Hat Tricks
    yr_c['HT'] = yr_c.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['HT_pg'] = yr_c.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Penalty Minutes
    yr_c['PIM'] = yr_c.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['PIM_pg'] = yr_c.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # PPG
    yr_c['PPG'] = yr_c.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['PPG_pg'] = yr_c.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SHG
    yr_c['SHG3'] = yr_c.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['SHG_pg'] = yr_c.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SOG
    yr_c['SOG'] = yr_c.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['SOG_pg'] = yr_c.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # BLocks
    yr_c['BLK'] = yr_c.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['BLK_pg'] = yr_c.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_c['FP'] = yr_c.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['FP_pg'] = yr_c.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    
    yr_c = yr_c.fillna(0)
    
    # DEFENSE
    # aggregate stats for last 3 games (sums and means)
    # goals
    yr_c['def_G3'] = yr_c.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_G_pg3'] = yr_c.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # assists
    yr_c['def_A3'] = yr_c.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_A_pg3'] = yr_c.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # points
    yr_c['def_PTS3'] = yr_c.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_PTS_pg3'] = yr_c.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # plusminus
    yr_c['def_plusminus3'] = yr_c.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_plusminus_pg3'] = yr_c.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Hat Tricks
    yr_c['def_HT3'] = yr_c.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_HT_pg3'] = yr_c.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Penalty Minutes
    yr_c['def_PIM3'] = yr_c.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_PIM_pg3'] = yr_c.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # PPG
    yr_c['def_PPG3'] = yr_c.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_PPG_pg3'] = yr_c.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SHG
    yr_c['def_SHG3'] = yr_c.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_SHG_pg3'] = yr_c.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SOG
    yr_c['def_SOG3'] = yr_c.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_SOG_pg3'] = yr_c.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # BLocks
    yr_c['def_BLK3'] = yr_c.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_BLK_pg3'] = yr_c.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_c['def_FP3'] = yr_c.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_c['def_FP_pg3'] = yr_c.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    
    # aggregate stats for season
    # goals
    yr_c['def_G'] = yr_c.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_G_pg'] = yr_c.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # assists
    yr_c['def_A'] = yr_c.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_A_pg'] = yr_c.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # points
    yr_c['def_PTS'] = yr_c.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_PTS_pg'] = yr_c.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # plusminus
    yr_c['def_PM'] = yr_c.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_PM_pg'] = yr_c.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Hat Tricks
    yr_c['def_HT'] = yr_c.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_HT_pg'] = yr_c.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Penalty Minutes
    yr_c['def_PIM'] = yr_c.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_PIM_pg'] = yr_c.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # PPG
    yr_c['def_PPG'] = yr_c.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_PPG_pg'] = yr_c.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SHG
    yr_c['def_SHG3'] = yr_c.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_SHG_pg'] = yr_c.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SOG
    yr_c['def_SOG'] = yr_c.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_SOG_pg'] = yr_c.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # BLocks
    yr_c['def_BLK'] = yr_c.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_BLK_pg'] = yr_c.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_c['def_FP'] = yr_c.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_c['def_FP_pg'] = yr_c.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    
    yr_c = yr_c.fillna(0)
    
    c_yr_dfs.append(yr_c)

    
c_stats = pd.concat(c_yr_dfs).drop_duplicates().reset_index(drop=True)
c_stats_all = c_stats.drop_duplicates(subset = ['Name', 'Date'])
c_stats_all.reset_index(drop=True, inplace = True)
c_stats_all.to_csv(etl_dir + 'c_stats.csv')
print('Done.')
    

2022
Done.


In [7]:
""" W ETL """
w_yr_dfs = []
for yr in range(2022, 2023): 
    print(yr)
    # filter to current year and PG position
    yr_w = player_stats[(player_stats['Season']==yr) & (player_stats['Position']=='LW') | (player_stats['Season']==yr) & (player_stats['Position']=='RW')].copy().reset_index(drop=True)
    yr_w = yr_w.drop_duplicates(subset = ['Name', 'Date'])
    yr_w.drop(['GoaltendingWins', 'GoaltendingLosses', 'GoaltendingOvertimeLosses', 'GoaltendingShotsAgainst', 'GoaltendingGoalsAgainst', 'GoaltendingSaves', 'GoaltendingShutouts',  'GoaltendingGoalsAgainstAverage', 'GoaltendingSavePercentage', 'GoaltendingMinutes'], axis = 1, inplace = True)

    # sort by players & week
    yr_w.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

    # aggregate stats for last 3 games (sums and means)
    # games
    yr_w['GM3'] = yr_w.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['GM_pg3'] = yr_w.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goals
    yr_w['G3'] = yr_w.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['G_pg3'] = yr_w.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # assists
    yr_w['A3'] = yr_w.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['A_pg3'] = yr_w.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # points
    yr_w['PTS3'] = yr_w.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['PTS_pg3'] = yr_w.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # plusminus
    yr_w['plusminus3'] = yr_w.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['plusminus_pg3'] = yr_w.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Hat Tricks
    yr_w['HT3'] = yr_w.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['HT_pg3'] = yr_w.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Penalty Minutes
    yr_w['PIM3'] = yr_w.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['PIM_pg3'] = yr_w.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # PPG
    yr_w['PPG3'] = yr_w.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['PPG_pg3'] = yr_w.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SHG
    yr_w['SHG3'] = yr_w.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['SHG_pg3'] = yr_w.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SOG
    yr_w['SOG3'] = yr_w.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['SOG_pg3'] = yr_w.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # BLocks
    yr_w['BLK3'] = yr_w.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['BLK_pg3'] = yr_w.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_w['FP3'] = yr_w.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['FP_pg3'] = yr_w.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    
    # aggregate stats for season
    # games
    yr_w['GM'] = yr_w.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    # goals
    yr_w['G'] = yr_w.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['G_pg'] = yr_w.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # assists
    yr_w['A'] = yr_w.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['A_pg'] = yr_w.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # points
    yr_w['PTS'] = yr_w.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['PTS_pg'] = yr_w.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # plusminus
    yr_w['PM'] = yr_w.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['PM_pg'] = yr_w.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Hat Tricks
    yr_w['HT'] = yr_w.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['HT_pg'] = yr_w.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Penalty Minutes
    yr_w['PIM'] = yr_w.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['PIM_pg'] = yr_w.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # PPG
    yr_w['PPG'] = yr_w.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['PPG_pg'] = yr_w.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SHG
    yr_w['SHG3'] = yr_w.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['SHG_pg'] = yr_w.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SOG
    yr_w['SOG'] = yr_w.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['SOG_pg'] = yr_w.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # BLocks
    yr_w['BLK'] = yr_w.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['BLK_pg'] = yr_w.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_w['FP'] = yr_w.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['FP_pg'] = yr_w.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    
    yr_w = yr_w.fillna(0)
    
    # DEFENSE
    # aggregate stats for last 3 games (sums and means)
    # goals
    yr_w['def_G3'] = yr_w.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_G_pg3'] = yr_w.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # assists
    yr_w['def_A3'] = yr_w.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_A_pg3'] = yr_w.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # points
    yr_w['def_PTS3'] = yr_w.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_PTS_pg3'] = yr_w.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # plusminus
    yr_w['def_plusminus3'] = yr_w.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_plusminus_pg3'] = yr_w.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Hat Tricks
    yr_w['def_HT3'] = yr_w.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_HT_pg3'] = yr_w.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Penalty Minutes
    yr_w['def_PIM3'] = yr_w.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_PIM_pg3'] = yr_w.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # PPG
    yr_w['def_PPG3'] = yr_w.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_PPG_pg3'] = yr_w.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SHG
    yr_w['def_SHG3'] = yr_w.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_SHG_pg3'] = yr_w.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SOG
    yr_w['def_SOG3'] = yr_w.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_SOG_pg3'] = yr_w.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # BLocks
    yr_w['def_BLK3'] = yr_w.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_BLK_pg3'] = yr_w.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_w['def_FP3'] = yr_w.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_w['def_FP_pg3'] = yr_w.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    
    # aggregate stats for season
    # goals
    yr_w['def_G'] = yr_w.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_G_pg'] = yr_w.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # assists
    yr_w['def_A'] = yr_w.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_A_pg'] = yr_w.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # points
    yr_w['def_PTS'] = yr_w.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_PTS_pg'] = yr_w.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # plusminus
    yr_w['def_PM'] = yr_w.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_PM_pg'] = yr_w.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Hat Tricks
    yr_w['def_HT'] = yr_w.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_HT_pg'] = yr_w.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Penalty Minutes
    yr_w['def_PIM'] = yr_w.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_PIM_pg'] = yr_w.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # PPG
    yr_w['def_PPG'] = yr_w.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_PPG_pg'] = yr_w.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SHG
    yr_w['def_SHG3'] = yr_w.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_SHG_pg'] = yr_w.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SOG
    yr_w['def_SOG'] = yr_w.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_SOG_pg'] = yr_w.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # BLocks
    yr_w['def_BLK'] = yr_w.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_BLK_pg'] = yr_w.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_w['def_FP'] = yr_w.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_w['def_FP_pg'] = yr_w.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    
    yr_w = yr_w.fillna(0)
    
    w_yr_dfs.append(yr_w)

    
w_stats = pd.concat(w_yr_dfs).drop_duplicates().reset_index(drop=True)
w_stats_all = w_stats.drop_duplicates(subset = ['Name', 'Date'])
w_stats_all.reset_index(drop=True, inplace = True)
w_stats_all.to_csv(etl_dir + 'w_stats.csv')
print('Done.')
    

2022
Done.


In [8]:
""" D ETL """
d_yr_dfs = []
for yr in range(2022, 2023): 
    print(yr)
    # filter to current year and PG position
    yr_d = player_stats[(player_stats['Season']==yr) & (player_stats['Position']=='D')].copy().reset_index(drop=True)
    yr_d = yr_d.drop_duplicates(subset = ['Name', 'Date'])
    yr_d.drop(['GoaltendingWins', 'GoaltendingLosses', 'GoaltendingOvertimeLosses', 'GoaltendingShotsAgainst', 'GoaltendingGoalsAgainst', 'GoaltendingSaves', 'GoaltendingShutouts',  'GoaltendingGoalsAgainstAverage', 'GoaltendingSavePercentage', 'GoaltendingMinutes'], axis = 1, inplace = True)

    # sort by players & week
    yr_d.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

    # aggregate stats for last 3 games (sums and means)
    # games
    yr_d['GM3'] = yr_d.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['GM_pg3'] = yr_d.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goals
    yr_d['G3'] = yr_d.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['G_pg3'] = yr_d.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # assists
    yr_d['A3'] = yr_d.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['A_pg3'] = yr_d.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # points
    yr_d['PTS3'] = yr_d.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['PTS_pg3'] = yr_d.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # plusminus
    yr_d['plusminus3'] = yr_d.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['plusminus_pg3'] = yr_d.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Hat Tricks
    yr_d['HT3'] = yr_d.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['HT_pg3'] = yr_d.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Penalty Minutes
    yr_d['PIM3'] = yr_d.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['PIM_pg3'] = yr_d.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # PPG
    yr_d['PPG3'] = yr_d.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['PPG_pg3'] = yr_d.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SHG
    yr_d['SHG3'] = yr_d.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['SHG_pg3'] = yr_d.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SOG
    yr_d['SOG3'] = yr_d.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['SOG_pg3'] = yr_d.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # BLocks
    yr_d['BLK3'] = yr_d.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['BLK_pg3'] = yr_d.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_d['FP3'] = yr_d.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['FP_pg3'] = yr_d.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    
    # aggregate stats for season
    # games
    yr_d['GM'] = yr_d.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    # goals
    yr_d['G'] = yr_d.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['G_pg'] = yr_d.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # assists
    yr_d['A'] = yr_d.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['A_pg'] = yr_d.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # points
    yr_d['PTS'] = yr_d.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['PTS_pg'] = yr_d.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # plusminus
    yr_d['PM'] = yr_d.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['PM_pg'] = yr_d.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Hat Tricks
    yr_d['HT'] = yr_d.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['HT_pg'] = yr_d.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Penalty Minutes
    yr_d['PIM'] = yr_d.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['PIM_pg'] = yr_d.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # PPG
    yr_d['PPG'] = yr_d.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['PPG_pg'] = yr_d.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SHG
    yr_d['SHG3'] = yr_d.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['SHG_pg'] = yr_d.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SOG
    yr_d['SOG'] = yr_d.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['SOG_pg'] = yr_d.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # BLocks
    yr_d['BLK'] = yr_d.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['BLK_pg'] = yr_d.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_d['FP'] = yr_d.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['FP_pg'] = yr_d.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    
    yr_d = yr_d.fillna(0)
    
    # DEFENSE
    # aggregate stats for last 3 games (sums and means)
    # goals
    yr_d['def_G3'] = yr_d.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_G_pg3'] = yr_d.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # assists
    yr_d['def_A3'] = yr_d.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_A_pg3'] = yr_d.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # points
    yr_d['def_PTS3'] = yr_d.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_PTS_pg3'] = yr_d.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # plusminus
    yr_d['def_plusminus3'] = yr_d.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_plusminus_pg3'] = yr_d.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Hat Tricks
    yr_d['def_HT3'] = yr_d.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_HT_pg3'] = yr_d.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # Penalty Minutes
    yr_d['def_PIM3'] = yr_d.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_PIM_pg3'] = yr_d.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # PPG
    yr_d['def_PPG3'] = yr_d.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_PPG_pg3'] = yr_d.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SHG
    yr_d['def_SHG3'] = yr_d.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_SHG_pg3'] = yr_d.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # SOG
    yr_d['def_SOG3'] = yr_d.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_SOG_pg3'] = yr_d.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # BLocks
    yr_d['def_BLK3'] = yr_d.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_BLK_pg3'] = yr_d.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_d['def_FP3'] = yr_d.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_d['def_FP_pg3'] = yr_d.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    
    # aggregate stats for season
    # goals
    yr_d['def_G'] = yr_d.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_G_pg'] = yr_d.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # assists
    yr_d['def_A'] = yr_d.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_A_pg'] = yr_d.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # points
    yr_d['def_PTS'] = yr_d.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_PTS_pg'] = yr_d.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # plusminus
    yr_d['def_PM'] = yr_d.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_PM_pg'] = yr_d.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Hat Tricks
    yr_d['def_HT'] = yr_d.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_HT_pg'] = yr_d.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # Penalty Minutes
    yr_d['def_PIM'] = yr_d.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_PIM_pg'] = yr_d.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # PPG
    yr_d['def_PPG'] = yr_d.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_PPG_pg'] = yr_d.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SHG
    yr_d['def_SHG3'] = yr_d.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_SHG_pg'] = yr_d.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # SOG
    yr_d['def_SOG'] = yr_d.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_SOG_pg'] = yr_d.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # BLocks
    yr_d['def_BLK'] = yr_d.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_BLK_pg'] = yr_d.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_d['def_FP'] = yr_d.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_d['def_FP_pg'] = yr_d.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    
    yr_d = yr_d.fillna(0)
    
    d_yr_dfs.append(yr_d)

    
d_stats = pd.concat(d_yr_dfs).drop_duplicates().reset_index(drop=True)
d_stats_all = d_stats.drop_duplicates(subset = ['Name', 'Date'])
d_stats_all.reset_index(drop=True, inplace = True)
d_stats_all.to_csv(etl_dir + 'd_stats.csv')
print('Done.')
    

2022
Done.


In [9]:
d_stats_all

,StatID,TeamID,PlayerID,Name,Team,Position,Games,Started,Season,Goals,...,def_PIM_pg,def_PPG,def_PPG_pg,def_SHG_pg,def_SOG,def_SOG_pg,def_BLK,def_BLK_pg,def_FP,def_FP_pg
0,2650187,8,30000239,Aaron Ekblad,FLA,D,0,0,2022,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,2655948,8,30000239,Aaron Ekblad,FLA,D,1,0,2022,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,2659117,8,30000239,Aaron Ekblad,FLA,D,0,0,2022,0.0,...,0.000000,0.0,0.000000,0.0,3.0,3.000000,1.0,1.000000,22.9,22.900000
3,2663787,8,30000239,Aaron Ekblad,FLA,D,1,0,2022,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,2665955,8,30000239,Aaron Ekblad,FLA,D,1,0,2022,1.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13398,2653169,5,30005771,Zachary Massicotte,OTT,D,0,0,2022,0.0,...,0.341463,0.0,0.000000,0.0,110.0,1.341463,75.0,0.914634,504.0,6.146341
13399,2650211,8,30005212,Zachary Uens,FLA,D,0,0,2022,0.0,...,0.451220,1.0,0.012195,0.0,70.0,0.853659,96.0,1.170732,440.1,5.367073
13400,2655972,8,30005212,Zachary Uens,FLA,D,0,0,2022,0.0,...,0.585366,2.0,0.024390,0.0,100.0,1.219512,85.0,1.036585,556.5,6.786585
13401,2659141,8,30005212,Zachary Uens,FLA,D,0,0,2022,0.0,...,0.585366,2.0,0.024390,0.0,99.0,1.207317,83.0,1.012195,543.7,6.630488


In [10]:
""" G ETL """
g_yr_gfs = []
for yr in range(2022, 2023): 
    print(yr)
    # filter to current year and PG position
    yr_g = player_stats[(player_stats['Season']==yr) & (player_stats['Position']=='G')].copy().reset_index(drop=True)
    yr_g = yr_g.drop_duplicates(subset = ['Name', 'Date'])
    yr_g.drop(['Goals', 'Assists', 'Points', 'PlusMinus', 'HatTricks',
       'PenaltyMinutes', 'PowerPlayGoals', 'ShortHandedGoals', 'ShotsOnGoal',
       'Blocks',], axis = 1, inplace = True)

    # sort by players & week
    yr_g.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

    # aggregate stats for last 3 games (sums and means)
    # games
    yr_g['GM3'] = yr_g.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GM_pg3'] = yr_g.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending wins
    yr_g['GTW3'] = yr_g.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTW_pg3'] = yr_g.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending losses
    yr_g['GTL3'] = yr_g.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTL_pg3'] = yr_g.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending overtime losses
    yr_g['GTOTL3'] = yr_g.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTOTL_pg3'] = yr_g.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending shots against
    yr_g['GTSA3'] = yr_g.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTSA_pg3'] = yr_g.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending goals against
    yr_g['GTGA3'] = yr_g.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTGA_pg3'] = yr_g.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # GoaltendingSaves
    yr_g['GTS3'] = yr_g.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTS_pg3'] = yr_g.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # GoaltendingShutouts
    yr_g['GTSO3'] = yr_g.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTSO_pg3'] = yr_g.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # GoaltendingGoalsAgainstAverage
    yr_g['GTGAA_pg3'] = yr_g.groupby(['Name'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
     # GoaltendingSavePercentage
    yr_g['GTSP_pg3'] = yr_g.groupby(['Name'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
     # GoaltendingMinutes
    yr_g['GTM3'] = yr_g.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['GTM_pg3'] = yr_g.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_g['FP3'] = yr_g.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['FP_pg3'] = yr_g.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # win percentage over last 3 games
    yr_g['WP3'] = yr_g['GTW3'] / yr_g['GM3']
    # saves per minute over last 3 games
    yr_g['SPM3'] = yr_g['GTS3'] / yr_g['GTM3']
    
    
    # aggregate stats for season
    # games
    yr_g['GM'] = yr_g.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    # goaltending wins
    yr_g['GTW'] = yr_g.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTW_pg'] = yr_g.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending losses
    yr_g['GTL'] = yr_g.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTL_pg'] = yr_g.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending overtime losses
    yr_g['GTOTL'] = yr_g.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTOTL_pg'] = yr_g.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending shots against
    yr_g['GTSA'] = yr_g.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTSA_pg'] = yr_g.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending goals against
    yr_g['GTGA'] = yr_g.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTGA_pg'] = yr_g.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # GoaltendingSaves
    yr_g['GTS'] = yr_g.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTS_pg'] = yr_g.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # GoaltendingShutouts
    yr_g['GTSO'] = yr_g.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTSO_pg'] = yr_g.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # GoaltendingGoalsAgainstAverage
    yr_g['GTGAA_pg'] = yr_g.groupby(['Name'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
     # GoaltendingSavePercentage
    yr_g['GTSP_pg'] = yr_g.groupby(['Name'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
     # GoaltendingMinutes
    yr_g['GTM'] = yr_g.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['GTM_pg'] = yr_g.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_g['FP'] = yr_g.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['FP_pg'] = yr_g.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # win percentage for season
    yr_g['WP'] = yr_g['GTW'] / yr_g['GM']
    # saves per minute for season
    yr_g['SPM'] = yr_g['GTS'] / yr_g['GTM']
    
    yr_g = yr_g.fillna(0)
    
    # DEFENSE
    # aggregate stats for last 3 games (sums and means)
    # games
    yr_g['def_GM3'] = yr_g.groupby(['Opponent'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GM_pg3'] = yr_g.groupby(['Opponent'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending wins
    yr_g['def_GTW3'] = yr_g.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTW_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending losses
    yr_g['def_GTL3'] = yr_g.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTL_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending overtime losses
    yr_g['def_GTOTL3'] = yr_g.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTOTL_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending shots against
    yr_g['def_GTSA3'] = yr_g.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTSA_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # goaltending goals against
    yr_g['def_GTGA3'] = yr_g.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTGA_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # GoaltendingSaves
    yr_g['def_GTS3'] = yr_g.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTS_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # GoaltendingShutouts
    yr_g['def_GTSO3'] = yr_g.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTSO_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # GoaltendingGoalsAgainstAverage
    yr_g['def_GTGAA_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
     # GoaltendingSavePercentage
    yr_g['def_GTSP_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
     # GoaltendingMinutes
    yr_g['def_GTM3'] = yr_g.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_GTM_pg3'] = yr_g.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # FP
    yr_g['def_FP3'] = yr_g.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
    yr_g['def_FP_pg3'] = yr_g.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
    # win percentage for last 3 games
    yr_g['def_WP3'] = yr_g['def_GTW3'] / yr_g['def_GM3']
    # saves per minute for last 3 games
    yr_g['def_SPM3'] = yr_g['def_GTS3'] / yr_g['def_GTM3']
    
    
    # aggregate stats for season
    # games
    yr_g['def_GM'] = yr_g.groupby(['Opponent'])['Games'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    # goaltending wins
    yr_g['def_GTW'] = yr_g.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTW_pg'] = yr_g.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending losses
    yr_g['def_GTL'] = yr_g.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTL_pg'] = yr_g.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending overtime losses
    yr_g['def_GTOTL'] = yr_g.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTOTL_pg'] = yr_g.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending shots against
    yr_g['def_GTSA'] = yr_g.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTSA_pg'] = yr_g.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # goaltending goals against
    yr_g['def_GTGA'] = yr_g.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTGA_pg'] = yr_g.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # GoaltendingSaves
    yr_g['def_GTS'] = yr_g.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTS_pg'] = yr_g.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # GoaltendingShutouts
    yr_g['def_GTSO'] = yr_g.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTSO_pg'] = yr_g.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # GoaltendingGoalsAgainstAverage
    yr_g['def_GTGAA_pg'] = yr_g.groupby(['Opponent'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
     # GoaltendingSavePercentage
    yr_g['def_GTSP_pg'] = yr_g.groupby(['Opponent'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
     # GoaltendingMinutes
    yr_g['def_GTM'] = yr_g.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_GTM_pg'] = yr_g.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # FP
    yr_g['def_FP'] = yr_g.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
    yr_g['def_FP_pg'] = yr_g.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
    # win percentage for last 3 games
    yr_g['def_WP'] = yr_g['def_GTW'] / yr_g['def_GM']
    # saves per minute for last 3 games
    yr_g['def_SPM'] = yr_g['def_GTS'] / yr_g['def_GTM']
    yr_g = yr_g.fillna(0)
    
    g_yr_gfs.append(yr_g)

    
g_stats = pd.concat(g_yr_gfs).drop_duplicates().reset_index(drop=True)
g_stats_all = g_stats.drop_duplicates(subset = ['Name', 'Date'])
g_stats_all.reset_index(drop=True, inplace = True)
g_stats_all.to_csv(etl_dir + 'g_stats.csv')
print('Done.')
    

2022
Done.


## Predictions

In [11]:
import os
import pickle
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from datetime import date
from datetime import datetime



working_directory = 'D:/machine_learning/DFS/NHL'
os.chdir(working_directory)
pred_dir = 'predictions/'
data_dir = 'CurrentData/'
etl_dir = 'CurrentData/ETL/'
os.chdir(working_directory)


# get today's date
today = date.today()

curr_date = str(today).replace('-','')

curr_season = 2022

player_stats = pd.read_csv('player_stats_current.csv', index_col = 0)
player_stats['Season'] = curr_season
sched = player_stats[['Season','Date','Team','Opponent']].drop_duplicates().reset_index(drop=True).rename(columns={'Opp':'Defense'})
sched = sched.sort_values(['Season','Date'], ascending = [False, False]).reset_index(drop=True)
sched.to_csv(data_dir + 'schedule_so_far.csv')

In [12]:
# get team abbreviations
player_stats['Team'].value_counts().sort_index().keys().tolist()

['ANA',
 'ARI',
 'BOS',
 'BUF',
 'CAR',
 'CBJ',
 'CGY',
 'CHI',
 'COL',
 'DAL',
 'DET',
 'EDM',
 'FLA',
 'LA',
 'MIN',
 'MON',
 'NAS',
 'NJ',
 'NYI',
 'NYR',
 'OTT',
 'PHI',
 'PIT',
 'SEA',
 'SJ',
 'STL',
 'TB',
 'TOR',
 'VAN',
 'VEG',
 'WAS',
 'WPG']

### Get Today's Schedule

In [13]:
import requests
from bs4 import BeautifulSoup
# CHANGE DAY AND DATE
# day of week retrieval
day_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
dt = datetime.now()
todays_day = day_dict[dt.weekday()]

# set date
str_date = str(today).replace('-', '')

# get this week's NBA schedule
sched_url = 'https://hashtaghockey.com/advanced-nhl-schedule-grid'

session_requests = requests.session()
r = session_requests.get(sched_url)
soup = BeautifulSoup(r.content, 'lxml')

rows = soup.findAll('tr')[1:]

l = []
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    
    # make sure that only rows with teams are added to the df
    if len(row[1]) == 1 and row[0] != '# Games Played':
        l.append(row)

# create df
columns =  [
    'Team',
    'Games',
    'Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday',
    'Saturday',
    'Sunday',
    'NextMonday',
    'NextTuesday'
    'NextWed',
    'NextThurs'
]

df = pd.DataFrame(l, columns = columns)

In [14]:
df

,Team,Games,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,NextMonday,NextTuesdayNextWed,NextThurs
0,Anaheim Ducks,4,,,,@DAL,@CHI,,,PIT,,@VGK
1,Arizona Coyotes,3,,,,MIN,,,,@CHI,@STL,
2,Boston Bruins,2,,,@TOR,,,,,,WSH,
3,Buffalo Sabres,2,,,CAR,,,,,,CGY,
4,Calgary Flames,3,,,,@NYR,,,@DET,,@BUF,
5,Carolina Hurricanes,3,,LAK,@BUF,,,,,,NYR,
6,Chicago Blackhawks,3,,,,,ANA,,,ARI,@WPJ,
7,Colorado Avalanche,3,,,,,@PIT,,@TBL,,@FLO,
8,Columbus Blue Jackets,3,,WSH,,,,,,TOR,@TOR,
9,Dallas Stars,3,,,,ANA,,MIN,,,TBL,


In [16]:
# remove @ symbols, denote which teams are off, and correct team abbreviations
df = df.replace({'\xa0':'off', 'FLO' : 'FLA', 'TBL': 'TB', 'VGK':'VEG', 'MTL' : 'MON', 'WPJ' : 'WPG', 'SJS' : 'SJ', 'LAK' : 'LA'}, regex=True)

df['Team'] = ['ANA',
             'ARI',
             'BOS',
             'BUF',
             'CGY',
             'CAR',
             'CHI',
             'COL',
             'CBJ',
             'DAL',
             'DET',
             'EDM',
             'FLA',
             'LA',
             'MIN',
             'MON',
             'NAS',
             'NJ',
             'NYI',
             'NYR',
             'OTT',
             'PHI',
             'PIT',
             'SJ',
             'SEA',
             'STL',
             'TB',
             'TOR',
             'VAN',
             'VEG',
             'WAS',
             'WPG']

# create subset dataframe with today's schedule
teams = df['Team']
defenses = df[todays_day]
# temporary fix for weird schedule
defenses = df['Thursday']
season = ['2022'] * len(teams)
dates = [str_date] * len(teams)

todays_df = pd.DataFrame({
    'Season' : season,
    'Date' : dates,
    'Team' : teams,
    'Opponent' : defenses})

print("Today's Schedule:")
display(todays_df)

sched_update = pd.concat([todays_df, sched], axis = 0)
sched_update.to_csv(data_dir + 'schedule_so_far_updated.csv')

Today's Schedule:


,Season,Date,Team,Opponent
0,2022,20230206,ANA,@DAL
1,2022,20230206,ARI,MIN
2,2022,20230206,BOS,off
3,2022,20230206,BUF,off
4,2022,20230206,CGY,@NYR
5,2022,20230206,CAR,off
6,2022,20230206,CHI,off
7,2022,20230206,COL,off
8,2022,20230206,CBJ,off
9,2022,20230206,DAL,ANA


In [18]:
# todays_df.sort_values('Team')['Team']

In [19]:
# todays_df.sort_values('Opponent')['Opponent']

In [17]:
import joblib

## C predictions (LGBM model)

In [222]:
working_directory = 'D:/machine_learning/DFS/NHL'
import os
os.chdir(working_directory)
pred_dir = 'predictions/'
data_dir = 'CurrentData/'
etl_dir = 'CurrentData/ETL/'

sched_update = pd.read_csv(data_dir + 'schedule_so_far_updated.csv', index_col = 0)
sched_update = sched_update[sched_update['Opponent'] != 'off']
sched_update['Season'] = sched_update['Season'].fillna(0)

# convert date from object dtype to datetime
sched_update['Date'] = pd.to_datetime(sched_update['Date'])

# convert date from datetime to string
sched_update['Date'] = sched_update['Date'].dt.strftime('%Y%m%d')
# sched_update[['Season','Date']] = sched_update[['Season','Date']].astype(int)

# drop duplicate players
players_df = player_stats[['Team','Name', 'Position']].drop_duplicates(subset = 'Name')

# df w/ schedule
curr_date_df = sched_update[(sched_update['Season']==curr_season) & (sched_update['Date']==curr_date)].copy().reset_index(drop=True)

# get players that play on the current date
curr_date_players = pd.merge(curr_date_df, players_df, how = 'left', on = ['Team'])

# get centers that play on the current date
curr_date_players_c = curr_date_players.loc[(curr_date_players['Position'] == 'C')]

# create HomeOrAway column for today's players since it would be nan otherwise
curr_date_players_c['HomeOrAway'] = np.where(curr_date_players_c['Opponent'].str.contains('@'), 'AWAY', 'HOME')

# c_vs is 2022 stats so far
c_vs = pd.read_csv(etl_dir + 'c_stats.csv', index_col = 0)
c_vs = c_vs.rename(columns={'Team_x' : 'Team'})
    
c_vs = c_vs[c_vs['Season']==curr_season].reset_index(drop=True)
curr_c_vs = pd.concat([c_vs, curr_date_players_c])
curr_c_vs.reset_index(inplace = True, drop = True)

curr_c_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

curr_c_vs = curr_c_vs.drop_duplicates(subset = ['Name', 'Date'])

# remove @ string from opponents
curr_c_vs = curr_c_vs.replace({'@':''}, regex=True)

# sort by players & week
curr_c_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

# aggregate stats for last 3 games (sums and means)
# games
curr_c_vs['GM3'] = curr_c_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['GM_pg3'] = curr_c_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goals
curr_c_vs['G3'] = curr_c_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['G_pg3'] = curr_c_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# assists
curr_c_vs['A3'] = curr_c_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['A_pg3'] = curr_c_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# points
curr_c_vs['PTS3'] = curr_c_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['PTS_pg3'] = curr_c_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# plusminus
curr_c_vs['plusminus3'] = curr_c_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['plusminus_pg3'] = curr_c_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Hat Tricks
curr_c_vs['HT3'] = curr_c_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['HT_pg3'] = curr_c_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Penalty Minutes
curr_c_vs['PIM3'] = curr_c_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['PIM_pg3'] = curr_c_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# PPG
curr_c_vs['PPG3'] = curr_c_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['PPG_pg3'] = curr_c_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SHG
curr_c_vs['SHG3'] = curr_c_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['SHG_pg3'] = curr_c_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SOG
curr_c_vs['SOG3'] = curr_c_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['SOG_pg3'] = curr_c_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# BLocks
curr_c_vs['BLK3'] = curr_c_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['BLK_pg3'] = curr_c_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_c_vs['FP3'] = curr_c_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['FP_pg3'] = curr_c_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())

# aggregate stats for season
# games
curr_c_vs['GM'] = curr_c_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
# goals
curr_c_vs['G'] = curr_c_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['G_pg'] = curr_c_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# assists
curr_c_vs['A'] = curr_c_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['A_pg'] = curr_c_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# points
curr_c_vs['PTS'] = curr_c_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['PTS_pg'] = curr_c_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# plusminus
curr_c_vs['PM'] = curr_c_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['PM_pg'] = curr_c_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Hat Tricks
curr_c_vs['HT'] = curr_c_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['HT_pg'] = curr_c_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Penalty Minutes
curr_c_vs['PIM'] = curr_c_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['PIM_pg'] = curr_c_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# PPG
curr_c_vs['PPG'] = curr_c_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['PPG_pg'] = curr_c_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SHG
curr_c_vs['SHG3'] = curr_c_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['SHG_pg'] = curr_c_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SOG
curr_c_vs['SOG'] = curr_c_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['SOG_pg'] = curr_c_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# BLocks
curr_c_vs['BLK'] = curr_c_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['BLK_pg'] = curr_c_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_c_vs['FP'] = curr_c_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['FP_pg'] = curr_c_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())

curr_c_vs = curr_c_vs.fillna(0)

# DEFENSE
# aggregate stats for last 3 games (sums and means)
# goals
curr_c_vs['def_G3'] = curr_c_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_G_pg3'] = curr_c_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# assists
curr_c_vs['def_A3'] = curr_c_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_A_pg3'] = curr_c_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# points
curr_c_vs['def_PTS3'] = curr_c_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_PTS_pg3'] = curr_c_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# plusminus
curr_c_vs['def_plusminus3'] = curr_c_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_plusminus_pg3'] = curr_c_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Hat Tricks
curr_c_vs['def_HT3'] = curr_c_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_HT_pg3'] = curr_c_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Penalty Minutes
curr_c_vs['def_PIM3'] = curr_c_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_PIM_pg3'] = curr_c_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# PPG
curr_c_vs['def_PPG3'] = curr_c_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_PPG_pg3'] = curr_c_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SHG
curr_c_vs['def_SHG3'] = curr_c_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_SHG_pg3'] = curr_c_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SOG
curr_c_vs['def_SOG3'] = curr_c_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_SOG_pg3'] = curr_c_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# BLocks
curr_c_vs['def_BLK3'] = curr_c_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_BLK_pg3'] = curr_c_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_c_vs['def_FP3'] = curr_c_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_c_vs['def_FP_pg3'] = curr_c_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())

# aggregate stats for season
# goals
curr_c_vs['def_G'] = curr_c_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_G_pg'] = curr_c_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# assists
curr_c_vs['def_A'] = curr_c_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_A_pg'] = curr_c_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# points
curr_c_vs['def_PTS'] = curr_c_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_PTS_pg'] = curr_c_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# plusminus
curr_c_vs['def_PM'] = curr_c_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_PM_pg'] = curr_c_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Hat Tricks
curr_c_vs['def_HT'] = curr_c_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_HT_pg'] = curr_c_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Penalty Minutes
curr_c_vs['def_PIM'] = curr_c_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_PIM_pg'] = curr_c_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# PPG
curr_c_vs['def_PPG'] = curr_c_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_PPG_pg'] = curr_c_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SHG
curr_c_vs['def_SHG3'] = curr_c_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_SHG_pg'] = curr_c_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SOG
curr_c_vs['def_SOG'] = curr_c_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_SOG_pg'] = curr_c_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# BLocks
curr_c_vs['def_BLK'] = curr_c_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_BLK_pg'] = curr_c_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_c_vs['def_FP'] = curr_c_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_c_vs['def_FP_pg'] = curr_c_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())

curr_c_vs = curr_c_vs.fillna(0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
curr_c_vs['HomeOrAway'] = encoder.fit_transform(curr_c_vs['HomeOrAway'].to_numpy().reshape(-1, 1))

# get features calculated for current date for each player
pred_df = curr_c_vs[curr_c_vs['Date']==curr_date].copy().reset_index(drop=True)

# creation of dataframe from which to predict FP
X_test = pred_df.copy()

# features for best model
c_features = ['HomeOrAway', 'GM3', 'GM_pg3', 'G3', 'G_pg3', 'A3', 'A_pg3', 'PTS3', 'PTS_pg3', 'plusminus3', 'plusminus_pg3', 'HT3', 'HT_pg3', 'PIM3', 'PIM_pg3', 'PPG3', 'PPG_pg3', 'SHG3', 'SHG_pg3', 'SOG3', 'SOG_pg3', 'BLK3', 'BLK_pg3', 'FP3', 'FP_pg3', 'GM', 'G', 'G_pg', 'A', 'A_pg', 'PTS', 'PTS_pg', 'PM', 'PM_pg', 'HT', 'HT_pg', 'PIM', 'PIM_pg', 'PPG', 'PPG_pg', 'SHG_pg', 'SOG', 'SOG_pg', 'BLK', 'BLK_pg', 'FP', 'FP_pg', 'def_G3', 'def_G_pg3', 'def_A3', 'def_A_pg3', 'def_PTS3', 'def_PTS_pg3', 'def_plusminus3', 'def_plusminus_pg3', 'def_HT3', 'def_HT_pg3', 'def_PIM3', 'def_PIM_pg3', 'def_PPG3', 'def_PPG_pg3', 'def_SHG3', 'def_SHG_pg3', 'def_SOG3', 'def_SOG_pg3', 'def_BLK3', 'def_BLK_pg3', 'def_FP3', 'def_FP_pg3', 'def_G', 'def_G_pg', 'def_A', 'def_A_pg', 'def_PTS', 'def_PTS_pg', 'def_PM', 'def_PM_pg', 'def_HT', 'def_HT_pg', 'def_PIM', 'def_PIM_pg', 'def_PPG', 'def_PPG_pg', 'def_SHG_pg', 'def_SOG', 'def_SOG_pg', 'def_BLK', 'def_BLK_pg', 'def_FP', 'def_FP_pg'] 

# open previously pickled scaler for G and load it
file = 'scalers/c_scaler_pt.pkl'
scaler = joblib.load(file)
    
file = 'scalers/c_X_train_pt.pkl'
C_X_train = joblib.load(file)

# fit scaler to guard X train df from when model was trained
c_scaler = scaler.fit(C_X_train[c_features])

# scale the prediction dataframe
X_test = pd.DataFrame(c_scaler.transform(X_test[c_features]), columns = X_test[c_features].columns)

# open best performing model, make predictions, write to csv
import joblib
filename = 'models/LGBM_models/C_model_allfeats_pt.pkl'
model = joblib.load(filename)

predictions = model.predict(X_test[c_features])
pred_df['Prediction_LGBM'] = predictions
pred_df['PredictRank_LGBM'] = pred_df.groupby(['Season','Date'])['Prediction_LGBM'].rank(method='min', ascending = False)

out_cols = ['Season','Date', 'Name', 'Team', 'Opponent', 'Prediction_LGBM', 'PredictRank_LGBM']
display(pred_df[out_cols].sort_values(by = 'PredictRank_LGBM'))
pred_df[out_cols].sort_values(by = 'PredictRank_LGBM').to_csv(pred_dir + 'C_Predictions_LGBM_'+str(curr_date)+'.csv')

C:\Users\Greg\AppData\Local\Temp/ipykernel_25740/2859704338.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_date_players_c['HomeOrAway'] = np.where(curr_date_players_c['Opponent'].str.contains('@'), 'AWAY', 'HOME')


,Season,Date,Name,Team,Opponent,Prediction_LGBM,PredictRank_LGBM
188,2022,20230206,Steven Stamkos,TB,FLA,17.090487,1.0
143,2022,20230206,Mika Zibanejad,NYR,CGY,16.947355,2.0
154,2022,20230206,Nico Hischier,NJ,VAN,16.141115,3.0
7,2022,20230206,Aleksander Barkov,FLA,TB,15.725858,4.0
38,2022,20230206,Carter Verhaeghe,FLA,TB,14.767676,5.0
...,...,...,...,...,...,...,...
141,2022,20230206,Max Willman,PHI,NYI,1.638380,209.0
158,2022,20230206,Olle Lycksell,PHI,NYI,1.617351,210.0
131,2022,20230206,Marco Rossi,MIN,ARI,1.535567,211.0
187,2022,20230206,Steven Fogarty,MIN,ARI,1.501497,212.0


## W predictions (RF model)

In [223]:
working_directory = 'D:/machine_learning/DFS/NHL'
import os
os.chdir(working_directory)
pred_dir = 'predictions/'
data_dir = 'CurrentData/'
etl_dir = 'CurrentData/ETL/'

sched_update = pd.read_csv(data_dir + 'schedule_so_far_updated.csv', index_col = 0)
sched_update = sched_update[sched_update['Opponent'] != 'off']
sched_update['Season'] = sched_update['Season'].fillna(0)

# convert date from object dtype to datetime
sched_update['Date'] = pd.to_datetime(sched_update['Date'])

# convert date from datetime to string
sched_update['Date'] = sched_update['Date'].dt.strftime('%Y%m%d')
# sched_update[['Season','Date']] = sched_update[['Season','Date']].astype(int)

# drop duplicate players
players_df = player_stats[['Team','Name', 'Position']].drop_duplicates(subset = 'Name')

# df w/ schedule
curr_date_df = sched_update[(sched_update['Season']==curr_season) & (sched_update['Date']==curr_date)].copy().reset_index(drop=True)

# get players that play on the current date
curr_date_players = pd.merge(curr_date_df, players_df, how = 'left', on = ['Team'])

# get wings that play on the current date
curr_date_players_w = curr_date_players.loc[(curr_date_players['Position'] == 'LW') | (curr_date_players['Position'] == 'RW')]

# create HomeOrAway column for today's players since it would be nan otherwise
curr_date_players_w['HomeOrAway'] = np.where(curr_date_players_w['Opponent'].str.contains('@'), 'AWAY', 'HOME')

# w_vs is 2022 stats so far
w_vs = pd.read_csv(etl_dir + 'w_stats.csv', index_col = 0)
w_vs = w_vs.rename(columns={'Team_x' : 'Team'})
    
w_vs = w_vs[w_vs['Season']==curr_season].reset_index(drop=True)
curr_w_vs = pd.concat([w_vs, curr_date_players_w])
curr_w_vs.reset_index(inplace = True, drop = True)

curr_w_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

curr_w_vs = curr_w_vs.drop_duplicates(subset = ['Name', 'Date'])

# remove @ string from opponents
curr_w_vs = curr_w_vs.replace({'@':''}, regex=True)

# sort by players & week
curr_w_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

# aggregate stats for last 3 games (sums and means)
# games
curr_w_vs['GM3'] = curr_w_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['GM_pg3'] = curr_w_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goals
curr_w_vs['G3'] = curr_w_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['G_pg3'] = curr_w_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# assists
curr_w_vs['A3'] = curr_w_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['A_pg3'] = curr_w_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# points
curr_w_vs['PTS3'] = curr_w_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['PTS_pg3'] = curr_w_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# plusminus
curr_w_vs['plusminus3'] = curr_w_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['plusminus_pg3'] = curr_w_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Hat Tricks
curr_w_vs['HT3'] = curr_w_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['HT_pg3'] = curr_w_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Penalty Minutes
curr_w_vs['PIM3'] = curr_w_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['PIM_pg3'] = curr_w_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# PPG
curr_w_vs['PPG3'] = curr_w_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['PPG_pg3'] = curr_w_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SHG
curr_w_vs['SHG3'] = curr_w_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['SHG_pg3'] = curr_w_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SOG
curr_w_vs['SOG3'] = curr_w_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['SOG_pg3'] = curr_w_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# BLocks
curr_w_vs['BLK3'] = curr_w_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['BLK_pg3'] = curr_w_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_w_vs['FP3'] = curr_w_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['FP_pg3'] = curr_w_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())

# aggregate stats for season
# games
curr_w_vs['GM'] = curr_w_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
# goals
curr_w_vs['G'] = curr_w_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['G_pg'] = curr_w_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# assists
curr_w_vs['A'] = curr_w_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['A_pg'] = curr_w_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# points
curr_w_vs['PTS'] = curr_w_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['PTS_pg'] = curr_w_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# plusminus
curr_w_vs['PM'] = curr_w_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['PM_pg'] = curr_w_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Hat Tricks
curr_w_vs['HT'] = curr_w_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['HT_pg'] = curr_w_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Penalty Minutes
curr_w_vs['PIM'] = curr_w_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['PIM_pg'] = curr_w_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# PPG
curr_w_vs['PPG'] = curr_w_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['PPG_pg'] = curr_w_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SHG
curr_w_vs['SHG3'] = curr_w_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['SHG_pg'] = curr_w_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SOG
curr_w_vs['SOG'] = curr_w_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['SOG_pg'] = curr_w_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# BLocks
curr_w_vs['BLK'] = curr_w_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['BLK_pg'] = curr_w_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_w_vs['FP'] = curr_w_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['FP_pg'] = curr_w_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())

curr_w_vs = curr_w_vs.fillna(0)

# DEFENSE
# aggregate stats for last 3 games (sums and means)
# goals
curr_w_vs['def_G3'] = curr_w_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_G_pg3'] = curr_w_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# assists
curr_w_vs['def_A3'] = curr_w_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_A_pg3'] = curr_w_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# points
curr_w_vs['def_PTS3'] = curr_w_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_PTS_pg3'] = curr_w_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# plusminus
curr_w_vs['def_plusminus3'] = curr_w_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_plusminus_pg3'] = curr_w_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Hat Tricks
curr_w_vs['def_HT3'] = curr_w_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_HT_pg3'] = curr_w_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Penalty Minutes
curr_w_vs['def_PIM3'] = curr_w_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_PIM_pg3'] = curr_w_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# PPG
curr_w_vs['def_PPG3'] = curr_w_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_PPG_pg3'] = curr_w_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SHG
curr_w_vs['def_SHG3'] = curr_w_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_SHG_pg3'] = curr_w_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SOG
curr_w_vs['def_SOG3'] = curr_w_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_SOG_pg3'] = curr_w_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# BLocks
curr_w_vs['def_BLK3'] = curr_w_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_BLK_pg3'] = curr_w_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_w_vs['def_FP3'] = curr_w_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_w_vs['def_FP_pg3'] = curr_w_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())

# aggregate stats for season
# goals
curr_w_vs['def_G'] = curr_w_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_G_pg'] = curr_w_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# assists
curr_w_vs['def_A'] = curr_w_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_A_pg'] = curr_w_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# points
curr_w_vs['def_PTS'] = curr_w_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_PTS_pg'] = curr_w_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# plusminus
curr_w_vs['def_PM'] = curr_w_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_PM_pg'] = curr_w_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Hat Tricks
curr_w_vs['def_HT'] = curr_w_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_HT_pg'] = curr_w_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Penalty Minutes
curr_w_vs['def_PIM'] = curr_w_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_PIM_pg'] = curr_w_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# PPG
curr_w_vs['def_PPG'] = curr_w_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_PPG_pg'] = curr_w_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SHG
curr_w_vs['def_SHG3'] = curr_w_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_SHG_pg'] = curr_w_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SOG
curr_w_vs['def_SOG'] = curr_w_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_SOG_pg'] = curr_w_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# BLocks
curr_w_vs['def_BLK'] = curr_w_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_BLK_pg'] = curr_w_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_w_vs['def_FP'] = curr_w_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_w_vs['def_FP_pg'] = curr_w_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())

curr_w_vs = curr_w_vs.fillna(0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
curr_w_vs['HomeOrAway'] = encoder.fit_transform(curr_w_vs['HomeOrAway'].to_numpy().reshape(-1, 1))


# get features calculated for current date for each player
pred_df = curr_w_vs[curr_w_vs['Date']==curr_date].copy().reset_index(drop=True)

# creation of dataframe from which to predict FP
X_test = pred_df.copy()

# features for best model
w_features = ['HomeOrAway', 'GM3', 'GM_pg3', 'G3', 'G_pg3', 'A3', 'A_pg3', 'PTS3', 'PTS_pg3', 'plusminus3', 'plusminus_pg3', 'HT3', 'HT_pg3', 'PIM3', 'PIM_pg3', 'PPG3', 'PPG_pg3', 'SHG3', 'SHG_pg3', 'SOG3', 'SOG_pg3', 'BLK3', 'BLK_pg3', 'FP3', 'FP_pg3', 'GM', 'G', 'G_pg', 'A', 'A_pg', 'PTS', 'PTS_pg', 'PM', 'PM_pg', 'HT', 'HT_pg', 'PIM', 'PIM_pg', 'PPG', 'PPG_pg', 'SHG_pg', 'SOG', 'SOG_pg', 'BLK', 'BLK_pg', 'FP', 'FP_pg', 'def_G3', 'def_G_pg3', 'def_A3', 'def_A_pg3', 'def_PTS3', 'def_PTS_pg3', 'def_plusminus3', 'def_plusminus_pg3', 'def_HT3', 'def_HT_pg3', 'def_PIM3', 'def_PIM_pg3', 'def_PPG3', 'def_PPG_pg3', 'def_SHG3', 'def_SHG_pg3', 'def_SOG3', 'def_SOG_pg3', 'def_BLK3', 'def_BLK_pg3', 'def_FP3', 'def_FP_pg3', 'def_G', 'def_G_pg', 'def_A', 'def_A_pg', 'def_PTS', 'def_PTS_pg', 'def_PM', 'def_PM_pg', 'def_HT', 'def_HT_pg', 'def_PIM', 'def_PIM_pg', 'def_PPG', 'def_PPG_pg', 'def_SHG_pg', 'def_SOG', 'def_SOG_pg', 'def_BLK', 'def_BLK_pg', 'def_FP', 'def_FP_pg'] 

# open previously pickled scaler for W and load it
file = 'scalers/w_scaler_pt.pkl'
scaler = joblib.load(file)
    
file = 'scalers/w_X_train_pt.pkl'
W_X_train = joblib.load(file)

# fit scaler to guard X train df from when model was trained
w_scaler = scaler.fit(W_X_train[w_features])

# scale the prediction dataframe
X_test = pd.DataFrame(w_scaler.transform(X_test[w_features]), columns = X_test[w_features].columns)

# open best performing model, make predictions, write to csv
import joblib
filename = 'models/LGBM_models/W_model_allfeats_pt.pkl'
model = joblib.load(filename)

predictions = model.predict(X_test[w_features])
pred_df['Prediction_LGBM'] = predictions
pred_df['PredictRank_LGBM'] = pred_df.groupby(['Season','Date'])['Prediction_LGBM'].rank(method='min', ascending = False)

out_cols = ['Season','Date', 'Name', 'Team', 'Opponent', 'Prediction_LGBM', 'PredictRank_LGBM']
display(pred_df[out_cols].sort_values(by = 'PredictRank_LGBM'))
pred_df[out_cols].sort_values(by = 'PredictRank_LGBM').to_csv(pred_dir + 'W_Predictions_LGBM_'+str(curr_date)+'.csv')

C:\Users\Greg\AppData\Local\Temp/ipykernel_25740/2665519919.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_date_players_w['HomeOrAway'] = np.where(curr_date_players_w['Opponent'].str.contains('@'), 'AWAY', 'HOME')


,Season,Date,Name,Team,Opponent,Prediction_LGBM,PredictRank_LGBM
114,2022,20230206,Jason Robertson,DAL,ANA,20.375277,1.0
165,2022,20230206,Matthew Tkachuk,FLA,TB,18.691894,2.0
28,2022,20230206,Artemi Panarin,NYR,CGY,18.167482,3.0
138,2022,20230206,Kirill Kaprizov,MIN,ARI,17.185094,4.0
192,2022,20230206,Nikita Kucherov,TB,FLA,14.557268,5.0
...,...,...,...,...,...,...,...
254,2022,20230206,Vasily Podkolzin,VAN,NJD,1.624371,263.0
217,2022,20230206,Radim Zohorna,CGY,NYR,1.536525,264.0
233,2022,20230206,Sammy Blais,NYR,CGY,1.388324,265.0
46,2022,20230206,Brett Ritchie,CGY,NYR,1.170925,266.0


## D predictions (RF model)

In [224]:
working_directory = 'D:/machine_learning/DFS/NHL'
import os
os.chdir(working_directory)
pred_dir = 'predictions/'
data_dir = 'CurrentData/'
etl_dir = 'CurrentData/ETL/'

sched_update = pd.read_csv(data_dir + 'schedule_so_far_updated.csv', index_col = 0)
sched_update = sched_update[sched_update['Opponent'] != 'off']
sched_update['Season'] = sched_update['Season'].fillna(0)

# convert date from object dtype to datetime
sched_update['Date'] = pd.to_datetime(sched_update['Date'])

# convert date from datetime to string
sched_update['Date'] = sched_update['Date'].dt.strftime('%Y%m%d')
# sched_update[['Season','Date']] = sched_update[['Season','Date']].astype(int)

# drop duplicate players
players_df = player_stats[['Team','Name', 'Position']].drop_duplicates(subset = 'Name')

# df w/ schedule
curr_date_df = sched_update[(sched_update['Season']==curr_season) & (sched_update['Date']==curr_date)].copy().reset_index(drop=True)

# get players that play on the current date
curr_date_players = pd.merge(curr_date_df, players_df, how = 'left', on = ['Team'])

# get defensemen that play on the current date
curr_date_players_d = curr_date_players.loc[curr_date_players['Position'] == 'D']

# create HomeOrAway column for today's players since it would be nan otherwise
curr_date_players_d['HomeOrAway'] = np.where(curr_date_players_d['Opponent'].str.contains('@'), 'AWAY', 'HOME')

# d_vs is 2022 stats so far
d_vs = pd.read_csv(etl_dir + 'd_stats.csv', index_col = 0)
d_vs = d_vs.rename(columns={'Team_x' : 'Team'})
    
d_vs = d_vs[d_vs['Season']==curr_season].reset_index(drop=True)
curr_d_vs = pd.concat([d_vs, curr_date_players_d])
curr_d_vs.reset_index(inplace = True, drop = True)

curr_d_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

curr_d_vs = curr_d_vs.drop_duplicates(subset = ['Name', 'Date'])

# remove @ string from opponents
curr_d_vs = curr_d_vs.replace({'@':''}, regex=True)

# sort by players & week
curr_d_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

# aggregate stats for last 3 games (sums and means)
# games
curr_d_vs['GM3'] = curr_d_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['GM_pg3'] = curr_d_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goals
curr_d_vs['G3'] = curr_d_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['G_pg3'] = curr_d_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# assists
curr_d_vs['A3'] = curr_d_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['A_pg3'] = curr_d_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# points
curr_d_vs['PTS3'] = curr_d_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['PTS_pg3'] = curr_d_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# plusminus
curr_d_vs['plusminus3'] = curr_d_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['plusminus_pg3'] = curr_d_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Hat Tricks
curr_d_vs['HT3'] = curr_d_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['HT_pg3'] = curr_d_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Penalty Minutes
curr_d_vs['PIM3'] = curr_d_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['PIM_pg3'] = curr_d_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# PPG
curr_d_vs['PPG3'] = curr_d_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['PPG_pg3'] = curr_d_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SHG
curr_d_vs['SHG3'] = curr_d_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['SHG_pg3'] = curr_d_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SOG
curr_d_vs['SOG3'] = curr_d_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['SOG_pg3'] = curr_d_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# BLocks
curr_d_vs['BLK3'] = curr_d_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['BLK_pg3'] = curr_d_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_d_vs['FP3'] = curr_d_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['FP_pg3'] = curr_d_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())

# aggregate stats for season
# games
curr_d_vs['GM'] = curr_d_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
# goals
curr_d_vs['G'] = curr_d_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['G_pg'] = curr_d_vs.groupby(['Name'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# assists
curr_d_vs['A'] = curr_d_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['A_pg'] = curr_d_vs.groupby(['Name'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# points
curr_d_vs['PTS'] = curr_d_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['PTS_pg'] = curr_d_vs.groupby(['Name'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# plusminus
curr_d_vs['PM'] = curr_d_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['PM_pg'] = curr_d_vs.groupby(['Name'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Hat Tricks
curr_d_vs['HT'] = curr_d_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['HT_pg'] = curr_d_vs.groupby(['Name'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Penalty Minutes
curr_d_vs['PIM'] = curr_d_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['PIM_pg'] = curr_d_vs.groupby(['Name'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# PPG
curr_d_vs['PPG'] = curr_d_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['PPG_pg'] = curr_d_vs.groupby(['Name'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SHG
curr_d_vs['SHG3'] = curr_d_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['SHG_pg'] = curr_d_vs.groupby(['Name'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SOG
curr_d_vs['SOG'] = curr_d_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['SOG_pg'] = curr_d_vs.groupby(['Name'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# BLocks
curr_d_vs['BLK'] = curr_d_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['BLK_pg'] = curr_d_vs.groupby(['Name'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_d_vs['FP'] = curr_d_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['FP_pg'] = curr_d_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())

curr_d_vs = curr_d_vs.fillna(0)

# DEFENSE
# aggregate stats for last 3 games (sums and means)
# goals
curr_d_vs['def_G3'] = curr_d_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_G_pg3'] = curr_d_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# assists
curr_d_vs['def_A3'] = curr_d_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_A_pg3'] = curr_d_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# points
curr_d_vs['def_PTS3'] = curr_d_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_PTS_pg3'] = curr_d_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# plusminus
curr_d_vs['def_plusminus3'] = curr_d_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_plusminus_pg3'] = curr_d_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Hat Tricks
curr_d_vs['def_HT3'] = curr_d_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_HT_pg3'] = curr_d_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# Penalty Minutes
curr_d_vs['def_PIM3'] = curr_d_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_PIM_pg3'] = curr_d_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# PPG
curr_d_vs['def_PPG3'] = curr_d_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_PPG_pg3'] = curr_d_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SHG
curr_d_vs['def_SHG3'] = curr_d_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_SHG_pg3'] = curr_d_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# SOG
curr_d_vs['def_SOG3'] = curr_d_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_SOG_pg3'] = curr_d_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# BLocks
curr_d_vs['def_BLK3'] = curr_d_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_BLK_pg3'] = curr_d_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_d_vs['def_FP3'] = curr_d_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_d_vs['def_FP_pg3'] = curr_d_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())

# aggregate stats for season
# goals
curr_d_vs['def_G'] = curr_d_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_G_pg'] = curr_d_vs.groupby(['Opponent'])['Goals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# assists
curr_d_vs['def_A'] = curr_d_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_A_pg'] = curr_d_vs.groupby(['Opponent'])['Assists'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# points
curr_d_vs['def_PTS'] = curr_d_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_PTS_pg'] = curr_d_vs.groupby(['Opponent'])['Points'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# plusminus
curr_d_vs['def_PM'] = curr_d_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_PM_pg'] = curr_d_vs.groupby(['Opponent'])['PlusMinus'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Hat Tricks
curr_d_vs['def_HT'] = curr_d_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_HT_pg'] = curr_d_vs.groupby(['Opponent'])['HatTricks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# Penalty Minutes
curr_d_vs['def_PIM'] = curr_d_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_PIM_pg'] = curr_d_vs.groupby(['Opponent'])['PenaltyMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# PPG
curr_d_vs['def_PPG'] = curr_d_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_PPG_pg'] = curr_d_vs.groupby(['Opponent'])['PowerPlayGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SHG
curr_d_vs['def_SHG3'] = curr_d_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_SHG_pg'] = curr_d_vs.groupby(['Opponent'])['ShortHandedGoals'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# SOG
curr_d_vs['def_SOG'] = curr_d_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_SOG_pg'] = curr_d_vs.groupby(['Opponent'])['ShotsOnGoal'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# BLocks
curr_d_vs['def_BLK'] = curr_d_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_BLK_pg'] = curr_d_vs.groupby(['Opponent'])['Blocks'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_d_vs['def_FP'] = curr_d_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_d_vs['def_FP_pg'] = curr_d_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())

curr_d_vs = curr_d_vs.fillna(0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
curr_d_vs['HomeOrAway'] = encoder.fit_transform(curr_d_vs['HomeOrAway'].to_numpy().reshape(-1, 1))

# get features calculated for current date for each player
pred_df = curr_d_vs[curr_d_vs['Date']==curr_date].copy().reset_index(drop=True)

# creation of dataframe from which to predict FP
X_test = pred_df.copy()

# features for best model
d_features = ['HomeOrAway', 'GM3', 'GM_pg3', 'G3', 'G_pg3', 'A3', 'A_pg3', 'PTS3', 'PTS_pg3', 'plusminus3', 'plusminus_pg3', 'HT3', 'HT_pg3', 'PIM3', 'PIM_pg3', 'PPG3', 'PPG_pg3', 'SHG3', 'SHG_pg3', 'SOG3', 'SOG_pg3', 'BLK3', 'BLK_pg3', 'FP3', 'FP_pg3', 'GM', 'G', 'G_pg', 'A', 'A_pg', 'PTS', 'PTS_pg', 'PM', 'PM_pg', 'HT', 'HT_pg', 'PIM', 'PIM_pg', 'PPG', 'PPG_pg', 'SHG_pg', 'SOG', 'SOG_pg', 'BLK', 'BLK_pg', 'FP', 'FP_pg', 'def_G3', 'def_G_pg3', 'def_A3', 'def_A_pg3', 'def_PTS3', 'def_PTS_pg3', 'def_plusminus3', 'def_plusminus_pg3', 'def_HT3', 'def_HT_pg3', 'def_PIM3', 'def_PIM_pg3', 'def_PPG3', 'def_PPG_pg3', 'def_SHG3', 'def_SHG_pg3', 'def_SOG3', 'def_SOG_pg3', 'def_BLK3', 'def_BLK_pg3', 'def_FP3', 'def_FP_pg3', 'def_G', 'def_G_pg', 'def_A', 'def_A_pg', 'def_PTS', 'def_PTS_pg', 'def_PM', 'def_PM_pg', 'def_HT', 'def_HT_pg', 'def_PIM', 'def_PIM_pg', 'def_PPG', 'def_PPG_pg', 'def_SHG_pg', 'def_SOG', 'def_SOG_pg', 'def_BLK', 'def_BLK_pg', 'def_FP', 'def_FP_pg'] 

# open previously pickled scaler for D and load it
file = 'scalers/d_scaler_pt.pkl'
scaler = joblib.load(file)
    
file = 'scalers/d_X_train_pt.pkl'
D_X_train = joblib.load(file)

# fit scaler to guard X train df from when model was trained
d_scaler = scaler.fit(D_X_train[d_features])

# scale the prediction dataframe
X_test = pd.DataFrame(d_scaler.transform(X_test[d_features]), columns = X_test[d_features].columns)

# open best performing model, make predictions, write to csv
import joblib
filename = 'models/LGBM_models/D_model_allfeats_pt.pkl'
model = joblib.load(filename)

predictions = model.predict(X_test[w_features])
pred_df['Prediction_LGBM'] = predictions
pred_df['PredictRank_LGBM'] = pred_df.groupby(['Season','Date'])['Prediction_LGBM'].rank(method='min', ascending = False)

out_cols = ['Season','Date', 'Name', 'Team', 'Opponent', 'Prediction_LGBM', 'PredictRank_LGBM']
display(pred_df[out_cols].sort_values(by = 'PredictRank_LGBM'))
pred_df[out_cols].sort_values(by = 'PredictRank_LGBM').to_csv(pred_dir + 'D_Predictions_LGBM_'+str(curr_date)+'.csv')

C:\Users\Greg\AppData\Local\Temp/ipykernel_25740/12145926.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_date_players_d['HomeOrAway'] = np.where(curr_date_players_d['Opponent'].str.contains('@'), 'AWAY', 'HOME')


,Season,Date,Name,Team,Opponent,Prediction_LGBM,PredictRank_LGBM
75,2022,20230206,Dougie Hamilton,NJ,VAN,17.477100,1.0
1,2022,20230206,Adam Fox,NYR,CGY,16.915779,2.0
118,2022,20230206,Jakob Chychrun,ARI,MIN,15.016674,3.0
185,2022,20230206,Miro Heiskanen,DAL,ANA,13.993833,4.0
26,2022,20230206,Brandon Montour,FLA,TB,12.500044,5.0
...,...,...,...,...,...,...,...
95,2022,20230206,Haydn Fleury,TB,FLA,0.936948,271.0
78,2022,20230206,Dysin Mayo,ARI,MIN,0.900850,272.0
10,2022,20230206,Andrej Sustr,MIN,ARI,0.872017,273.0
188,2022,20230206,Nick DeSimone,CGY,NYR,0.826435,274.0


## G predictions

In [226]:
working_directory = 'D:/machine_learning/DFS/NHL'
import os
os.chdir(working_directory)
pred_dir = 'predictions/'
data_dir = 'CurrentData/'
etl_dir = 'CurrentData/ETL/'

sched_update = pd.read_csv(data_dir + 'schedule_so_far_updated.csv', index_col = 0)
sched_update = sched_update[sched_update['Opponent'] != 'off']
sched_update['Season'] = sched_update['Season'].fillna(0)

# convert date from object dtype to datetime
sched_update['Date'] = pd.to_datetime(sched_update['Date'])

# convert date from datetime to string
sched_update['Date'] = sched_update['Date'].dt.strftime('%Y%m%d')
# sched_update[['Season','Date']] = sched_update[['Season','Date']].astype(int)

# drop duplicate players
players_df = player_stats[['Team','Name', 'Position']].drop_duplicates(subset = 'Name')

# df w/ schedule
curr_date_df = sched_update[(sched_update['Season']==curr_season) & (sched_update['Date']==curr_date)].copy().reset_index(drop=True)

# get players that play on the current date
curr_date_players = pd.merge(curr_date_df, players_df, how = 'left', on = ['Team'])

# get centers that play on the current date
curr_date_players_g = curr_date_players.loc[curr_date_players['Position'] == 'G']

# create HomeOrAway column for today's players since it would be nan otherwise
curr_date_players_g['HomeOrAway'] = np.where(curr_date_players_g['Opponent'].str.contains('@'), 'AWAY', 'HOME')

# g_vs is 2022 stats so far
g_vs = pd.read_csv(etl_dir + 'g_stats.csv', index_col = 0)
g_vs = g_vs.rename(columns={'Team_x' : 'Team'})
    
g_vs = g_vs[g_vs['Season']==curr_season].reset_index(drop=True)
curr_g_vs = pd.concat([g_vs, curr_date_players_g])
curr_g_vs.reset_index(inplace = True, drop = True)

curr_g_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

curr_g_vs = curr_g_vs.drop_duplicates(subset = ['Name', 'Date'])

# remove @ string from opponents
curr_g_vs = curr_g_vs.replace({'@':''}, regex=True)

# sort by players & week
curr_g_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

# sort by players & week
curr_g_vs.sort_values(['Name', 'Date'], ascending = [True, True], inplace = True)

# aggregate stats for last 3 games (sums and means)
# games
curr_g_vs['GM3'] = curr_g_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GM_pg3'] = curr_g_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending wins
curr_g_vs['GTW3'] = curr_g_vs.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTW_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending losses
curr_g_vs['GTL3'] = curr_g_vs.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTL_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending overtime losses
curr_g_vs['GTOTL3'] = curr_g_vs.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTOTL_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending shots against
curr_g_vs['GTSA3'] = curr_g_vs.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTSA_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending goals against
curr_g_vs['GTGA3'] = curr_g_vs.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTGA_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingSaves
curr_g_vs['GTS3'] = curr_g_vs.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTS_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingShutouts
curr_g_vs['GTSO3'] = curr_g_vs.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTSO_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingGoalsAgainstAverage
curr_g_vs['GTGAA_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingSavePercentage
curr_g_vs['GTSP_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingMinutes
curr_g_vs['GTM3'] = curr_g_vs.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['GTM_pg3'] = curr_g_vs.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_g_vs['FP3'] = curr_g_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['FP_pg3'] = curr_g_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# win percentage over last 3 games
curr_g_vs['WP3'] = curr_g_vs['GTW3'] / curr_g_vs['GM3']
# saves per minute over last 3 games
curr_g_vs['SPM3'] = curr_g_vs['GTS3'] / curr_g_vs['GTM3']


# aggregate stats for season
# games
curr_g_vs['GM'] = curr_g_vs.groupby(['Name'])['Games'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
# goaltending wins
curr_g_vs['GTW'] = curr_g_vs.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTW_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending losses
curr_g_vs['GTL'] = curr_g_vs.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTL_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending overtime losses
curr_g_vs['GTOTL'] = curr_g_vs.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTOTL_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending shots against
curr_g_vs['GTSA'] = curr_g_vs.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTSA_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending goals against
curr_g_vs['GTGA'] = curr_g_vs.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTGA_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingSaves
curr_g_vs['GTS'] = curr_g_vs.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTS_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingShutouts
curr_g_vs['GTSO'] = curr_g_vs.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTSO_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingGoalsAgainstAverage
curr_g_vs['GTGAA_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingSavePercentage
curr_g_vs['GTSP_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingMinutes
curr_g_vs['GTM'] = curr_g_vs.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['GTM_pg'] = curr_g_vs.groupby(['Name'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_g_vs['FP'] = curr_g_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['FP_pg'] = curr_g_vs.groupby(['Name'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# win percentage for season
curr_g_vs['WP'] = curr_g_vs['GTW'] / curr_g_vs['GM']
# saves per minute for season
curr_g_vs['SPM'] = curr_g_vs['GTS'] / curr_g_vs['GTM']

curr_g_vs = curr_g_vs.fillna(0)

# DEFENSE
# aggregate stats for last 3 games (sums and means)
# games
curr_g_vs['def_GM3'] = curr_g_vs.groupby(['Opponent'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GM_pg3'] = curr_g_vs.groupby(['Opponent'])['Games'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending wins
curr_g_vs['def_GTW3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTW_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending losses
curr_g_vs['def_GTL3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTL_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending overtime losses
curr_g_vs['def_GTOTL3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTOTL_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending shots against
curr_g_vs['def_GTSA3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTSA_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# goaltending goals against
curr_g_vs['def_GTGA3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTGA_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingSaves
curr_g_vs['def_GTS3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTS_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingShutouts
curr_g_vs['def_GTSO3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTSO_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingGoalsAgainstAverage
curr_g_vs['def_GTGAA_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingSavePercentage
curr_g_vs['def_GTSP_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# GoaltendingMinutes
curr_g_vs['def_GTM3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_GTM_pg3'] = curr_g_vs.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# FP
curr_g_vs['def_FP3'] = curr_g_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).sum())
curr_g_vs['def_FP_pg3'] = curr_g_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(3, min_periods = 1).mean())
# win percentage for last 3 games
curr_g_vs['def_WP3'] = curr_g_vs['def_GTW3'] / curr_g_vs['def_GM3']
# saves per minute for last 3 games
curr_g_vs['def_SPM3'] = curr_g_vs['def_GTS3'] / curr_g_vs['def_GTM3']


# aggregate stats for season
# games
curr_g_vs['def_GM'] = curr_g_vs.groupby(['Opponent'])['Games'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
# goaltending wins
curr_g_vs['def_GTW'] = curr_g_vs.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTW_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingWins'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending losses
curr_g_vs['def_GTL'] = curr_g_vs.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTL_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending overtime losses
curr_g_vs['def_GTOTL'] = curr_g_vs.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTOTL_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingOvertimeLosses'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending shots against
curr_g_vs['def_GTSA'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTSA_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShotsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# goaltending goals against
curr_g_vs['def_GTGA'] = curr_g_vs.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTGA_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingGoalsAgainst'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingSaves
curr_g_vs['def_GTS'] = curr_g_vs.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTS_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingSaves'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingShutouts
curr_g_vs['def_GTSO'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTSO_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingShutouts'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingGoalsAgainstAverage
curr_g_vs['def_GTGAA_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingGoalsAgainstAverage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingSavePercentage
curr_g_vs['def_GTSP_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingSavePercentage'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# GoaltendingMinutes
curr_g_vs['def_GTM'] = curr_g_vs.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_GTM_pg'] = curr_g_vs.groupby(['Opponent'])['GoaltendingMinutes'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# FP
curr_g_vs['def_FP'] = curr_g_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).sum())
curr_g_vs['def_FP_pg'] = curr_g_vs.groupby(['Opponent'])['FantasyPointsFanDuel'].transform(lambda x: x.shift().rolling(82, min_periods = 1).mean())
# win percentage for last 3 games
curr_g_vs['def_WP'] = curr_g_vs['def_GTW'] / curr_g_vs['def_GM']
# saves per minute for last 3 games
curr_g_vs['def_SPM'] = curr_g_vs['def_GTS'] / curr_g_vs['def_GTM']
curr_g_vs = curr_g_vs.fillna(0)

# ordinal encode HomeorAway column
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
curr_g_vs['HomeOrAway'] = encoder.fit_transform(curr_g_vs['HomeOrAway'].to_numpy().reshape(-1, 1))

# get features calculated for current date for each player
pred_df = curr_g_vs[curr_g_vs['Date']==curr_date].copy().reset_index(drop=True)

# creation of dataframe from which to predict FP
X_test = pred_df.copy()

# features for best model (50 features model)
g_features = ['SPM', 'def_SPM', 'def_GTL_pg', 'def_GTGAA_pg', 'GTGAA_pg', 'SPM3', 'GTSP_pg', 'def_GTGAA_pg3', 'def_SPM3', 'def_WP', 'GTL_pg', 'def_GTSO_pg', 'def_GTGA_pg', 'GTOTL_pg', 'def_GTOTL_pg', 'def_GTSP_pg', 'GTSO_pg', 'GTGA_pg', 'GTGAA_pg3', 'FP_pg', 'WP', 'GTW_pg', 'GTM_pg', 'def_GTGA', 'def_FP_pg', 'def_GTSP_pg3', 'GTM3', 'GTSP_pg3', 'FP3', 'def_GTW_pg', 'GTM_pg3', 'def_GTM_pg', 'FP', 'def_FP', 'GTSA_pg', 'def_GTSA_pg', 'GTW', 'FP_pg3', 'GTGA', 'GTS_pg', 'GTS_pg3', 'def_GTS_pg', 'GTSA3', 'def_FP_pg3', 'def_FP3', 'GTS3', 'HomeOrAway', 'GTL', 'GTSA_pg3', 'def_GTW']

# open previously pickled scaler for G and load it
file = 'scalers/g_scaler_pt.pkl'
scaler = joblib.load(file)
    
file = 'scalers/g_X_train_pt.pkl'
G_X_train = joblib.load(file)

# fit scaler to goalie X train df from when model was trained
g_scaler = scaler.fit(G_X_train[g_features])

# scale the prediction dataframe
X_test = pd.DataFrame(g_scaler.transform(X_test[g_features]), columns = X_test[g_features].columns)

# open best performing model, make predictions, write to csv
import joblib
filename = 'models/LGBM_models/G_model_50feats_pt.pkl'
model = joblib.load(filename)

predictions = model.predict(X_test[g_features])
pred_df['Prediction_LGBM'] = predictions
pred_df['PredictRank_LGBM'] = pred_df.groupby(['Season','Date'])['Prediction_LGBM'].rank(method='min', ascending = False)

# average predictions for LGBM and LGBM models, store in prediction LGBM
pred_df['Prediction_LGBM'] = (pred_df['Prediction_LGBM'] + pred_df['Prediction_LGBM'])/2

out_cols = ['Season','Date', 'Name', 'Team', 'Opponent', 'Prediction_LGBM', 'PredictRank_LGBM']
display(pred_df[out_cols].sort_values(by = 'PredictRank_LGBM'))
pred_df[out_cols].sort_values(by = 'PredictRank_LGBM').to_csv(pred_dir + 'G_Predictions_LGBM_'+str(curr_date)+'.csv')

C:\Users\Greg\AppData\Local\Temp/ipykernel_25740/655233280.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_date_players_g['HomeOrAway'] = np.where(curr_date_players_g['Opponent'].str.contains('@'), 'AWAY', 'HOME')


,Season,Date,Name,Team,Opponent,Prediction_LGBM,PredictRank_LGBM
54,2022,20230206,Marc-Andre Fleury,MIN,ARI,16.860230,1.0
37,2022,20230206,Jake Oettinger,DAL,ANA,16.579115,2.0
83,2022,20230206,Vitek Vanecek,NJ,VAN,16.103930,3.0
16,2022,20230206,Carter Hart,PHI,NYI,15.511690,4.0
32,2022,20230206,Igor Shesterkin,NYR,CGY,14.899326,5.0
...,...,...,...,...,...,...,...
2,2022,20230206,Aku Koskenvuo,VAN,NJD,2.113114,78.0
81,2022,20230206,Tyler Muszelik,FLA,TB,2.080880,83.0
48,2022,20230206,Ken Appleby,NYI,PHI,1.940632,84.0
0,2022,20230206,Adam Scheel,DAL,ANA,1.790218,85.0


# Part 5: Lineup Optimization

## Slate Lineups

In [22]:
import os
import pickle
import numpy as np
import pandas as pd

pred_dir = 'Predictions/'
roster_file = 'rosters/MIN_ARI.csv'

In [23]:
# read in model predictions
c_preds = pd.read_csv(pred_dir + 'C_Predictions_LGBM_'+str(curr_date)+'.csv')
w_preds = pd.read_csv(pred_dir + 'W_Predictions_LGBM_'+str(curr_date)+'.csv')
d_preds = pd.read_csv(pred_dir + 'D_Predictions_LGBM_'+str(curr_date)+'.csv')
g_preds = pd.read_csv(pred_dir + 'G_Predictions_LGBM_'+str(curr_date)+'.csv')

# concat predictions to a single dataframe
all_preds = pd.concat([c_preds, w_preds, d_preds, g_preds]).reset_index(drop = True)
all_preds = all_preds.drop(['Unnamed: 0'], axis = 1)

In [24]:
all_preds = all_preds.rename(columns = {'Name' : 'Nickname'})

# fix names with accent marks/suffixes
import unidecode

for i in range(len(all_preds)):
    # accent mark fix
    all_preds.iloc[i, 2] = unidecode.unidecode(all_preds.iloc[i, 2])
    # drop Jrs.
    if all_preds.iloc[i, 2][-3:] == 'Jr.' or all_preds.iloc[i, 2][-2:] == 'IV':
        all_preds.iloc[i, 2] = all_preds.iloc[i, 2][:-3]

# read in fanduel salaries
fd_df = pd.read_csv(roster_file)

In [25]:
# merge predictions and FD df so salaries are available
all_preds_merged = pd.merge(all_preds, fd_df, on='Nickname')

# drop players w/ 0 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] != 0)].reset_index(drop = True)

# keep players w/ > 2.1 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] >= 2.1)].reset_index(drop = True)

# drop injured players
all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'O')].reset_index(drop = True)
all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'IR')].reset_index(drop = True)
# all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'NA')].reset_index(drop = True)
# all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'GTD')].reset_index(drop = True)

# drop players based on how many games they've played
# makes sure that players who don't play aren't considered
all_preds_merged = all_preds_merged[(all_preds_merged.Played >= all_preds_merged.Played.quantile(.2))].reset_index(drop = True)

# drop players based on salary cutoffs
# makes sure that bad players are excluded for each position
all_preds_merged = all_preds_merged[(all_preds_merged.Salary >= 4000)].reset_index(drop = True)

# drop unnamed columns originating from fd_df
cols = all_preds_merged.columns
for col in cols:
    if 'Unnamed'in col:
        all_preds_merged.drop(col, inplace = True, axis = 1)

In [26]:
all_preds_merged

,Season,Date,Nickname,Team_x,Opponent_x,Prediction_LGBM,PredictRank_LGBM,Id,Position,First Name,...,FPPG,Played,Salary,Game,Team_y,Opponent_y,Injury Indicator,Injury Details,Tier,Roster Position
0,2022,20230206,Joel Eriksson Ek,MIN,ARI,12.282969,17.0,86682-73949,C,Joel,...,14.458333,48.0,13000,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
1,2022,20230206,Nick Schmaltz,ARI,MIN,8.688095,30.0,86682-73889,W,Nick,...,10.668421,38.0,10000,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
2,2022,20230206,Barrett Hayton,ARI,MIN,7.974975,35.0,86682-97108,C,Barrett,...,7.000000,50.0,7500,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
3,2022,20230206,Frederick Gaudreau,MIN,ARI,7.923683,38.0,86682-59209,W,Frederick,...,8.618750,48.0,9500,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
4,2022,20230206,Nick Bjugstad,ARI,MIN,7.808409,39.0,86682-39249,C,Nick,...,7.864000,50.0,9000,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
5,2022,20230206,Sam Steel,MIN,ARI,7.220636,44.0,86682-70928,C,Sam,...,6.931250,48.0,7500,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
6,2022,20230206,Travis Boyd,ARI,MIN,6.279988,53.0,86682-62275,C,Travis,...,6.194000,50.0,7000,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
7,2022,20230206,Connor Dewar,MIN,ARI,6.044494,56.0,86682-97056,W,Connor,...,5.463830,47.0,7000,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
8,2022,20230206,Jack McBain,ARI,MIN,5.308228,65.0,86682-167209,C,Jack,...,5.024000,50.0,6500,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
9,2022,20230206,Mason Shaw,MIN,ARI,2.610430,195.0,86682-82122,C,Mason,...,6.517647,34.0,7500,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL


In [27]:
# players by position
all_preds_merged['Position'].value_counts()

W    16
D    15
C     8
Name: Position, dtype: int64

In [28]:
# filter_df for pulp
filter_df = all_preds_merged.copy()
filter_df['MaxUsed'] = 0
filter_df['Pos_C'] = np.where(filter_df['Position']=='C', 1, 0)
filter_df['Pos_W'] = np.where(filter_df['Position']=='W', 1, 0)
filter_df['Pos_D'] = np.where(filter_df['Position']=='D', 1, 0)
filter_df['Pos_G'] = np.where(filter_df['Position']=='G', 1, 0)
filter_df['Pos_All'] = 1

In [29]:
encoded = pd.get_dummies(filter_df['Nickname']) #<-- One-Hot Encoding 

In [30]:
filter_df = filter_df.join(encoded) #<-- joining it to the raw_data table

In [31]:
encoded.columns.tolist()

['Alex Chiasson',
 'Alex Goligoski',
 'Andrej Sustr',
 'Barrett Hayton',
 'Brandon Duhaime',
 'Calen Addison',
 'Cam Dineen',
 'Christian Fischer',
 'Clayton Keller',
 'Connor Dewar',
 'Dylan Guenther',
 'Frederick Gaudreau',
 'J.J. Moser',
 'Jack McBain',
 'Jakob Chychrun',
 'Jared Spurgeon',
 'Joel Eriksson Ek',
 'Jon Merrill',
 'Jonas Brodin',
 'Jordan Greenway',
 'Josh Brown',
 'Juuso Valimaki',
 'Kirill Kaprizov',
 'Lawson Crouse',
 'Marcus Foligno',
 'Mason Shaw',
 'Matias Maccelli',
 'Mats Zuccarello',
 'Matt Dumba',
 'Nick Bjugstad',
 'Nick Ritchie',
 'Nick Schmaltz',
 'Patrik Nemeth',
 'Ryan Hartman',
 'Ryan Reaves',
 'Sam Steel',
 'Travis Boyd',
 'Troy Stecher',
 'Vladislav Kolyachonok']

In [32]:
import pulp

def pred_lineup(df, lineups, overlap, player_names = [0], nonplayer_names = [0]):
    #Initialize our PuLP problem, we choose pulp.LpMinimize because we are going to want the lowest ranked players numerically. (1st is better than 2nd, technically smaller though)
    prob = pulp.LpProblem('NHL_DK', pulp.LpMaximize)
    
    #Count How Many Different Players We Have (Including Team D)
    num_all = df.shape[0]
    
    # get unique teams
    uniq_teams = df['Team_x'].unique().tolist()
    
    # one-hot encoding for teams
    team_ohe = pd.get_dummies(filter_df['Team_x'])
    df = df.join(team_ohe)
    
    #Create Salary Cap Variable
    salary_cap = 55000
    
    #Create List Of PuLP player variables
    players_lineup = [pulp.LpVariable("Player_{}".format(i+1), cat="Binary") for i in range(num_all)]
    
    #Tell our PuLP solver that we want our lineup to have our prediction field to be maximized
    prob += pulp.lpSum((pulp.lpSum(df.loc[i, 'Prediction_LGBM'] * players_lineup[i] for i in range(num_all))))
    
    # Make sure only 2 Cs are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_C'] * players_lineup[i] for i in range(num_all)))  >= 2)
    prob += ((pulp.lpSum(df.loc[i, 'Pos_C'] * players_lineup[i] for i in range(num_all)))  <= 3)
    
    # Make sure only 2 Ws are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_W'] * players_lineup[i] for i in range(num_all)))  >= 2)
    prob += ((pulp.lpSum(df.loc[i, 'Pos_W'] * players_lineup[i] for i in range(num_all)))  <= 3)
    
    # Make sure only 2 Ds are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_D'] * players_lineup[i] for i in range(num_all)))  >= 2)
    prob += ((pulp.lpSum(df.loc[i, 'Pos_D'] * players_lineup[i] for i in range(num_all)))  <= 3)
    
    # Make sure only 1 G is selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_G'] * players_lineup[i] for i in range(num_all)))  == 1)
    
    #Make Sure To Select 9 Players Total
    prob += ((pulp.lpSum(df.loc[i, 'Pos_All'] * players_lineup[i] for i in range(num_all))) == 9)
    
    #add the salary constraint
    prob += ((pulp.lpSum(df.loc[i, 'Salary'] * players_lineup[i] for i in range(num_all))) <= salary_cap)
    
    #Add Our MaxUsed Constraint
    prob += ((pulp.lpSum(df.loc[i, 'MaxUsed'] * players_lineup[i] for i in range(num_all))) == 0)
    
    # constraint to not select more than one player from a team
    for team in uniq_teams:
        prob += ((pulp.lpSum(df.loc[i, team] * players_lineup[i] for i in range(num_all)))  <= 4)
    
    #select specific player, if indicated
    if player_names != [0]:
        for player_name in player_names:
            prob += ((pulp.lpSum(df.loc[i, player_name] * players_lineup[i] for i in range(num_all)))  == 1)
            
    #remove specific player from consideration, if indicated
    if nonplayer_names != [0]:
        for nonplayer_name in nonplayer_names:
            prob += ((pulp.lpSum(df.loc[i, nonplayer_name] * players_lineup[i] for i in range(num_all)))  == 0)
    
    # add constraint to select only 1 of each player
    for player in encoded.columns.tolist():
        prob += ((pulp.lpSum(df.loc[i, player] * players_lineup[i] for i in range(num_all))) <= 1)
    
    #variance constraints - each lineup can't have more than the num overlap of any combination of players in any previous lineups
    for i in range(len(lineups)):
        prob += ((pulp.lpSum(lineups[i][k] * players_lineup[k] for k in range(num_all))) <= overlap)
        
    #solve the problem
    status = prob.solve()
    
    #check if the optimizer found an optimal solution
    if status != pulp.LpStatusOptimal:
        print('Only {} feasible lineups produced'.format(len(lineups)), '\n')
        return None
    lineup_copy = []
    for i in range(num_all):
        if players_lineup[i].varValue >= 0.9 and players_lineup[i].varValue <= 1.1:
            lineup_copy.append(1)
        else:
            lineup_copy.append(0)

    return lineup_copy

In [33]:
# function to tranform lineups resulting from PULP optimization
def transform_lineup(lineup, lineup_num):
    positions, names, ids, salaries = [], [], [], []
    game_infos, teams, ppgs, ranks = [], [], [], []
    predictions = []
    pos_C = []
    pos_W = []
    pos_D = []
    pos_G = []

    players_lineup = lineup[:len(lineup)]
    for num, player in enumerate(players_lineup):
        if player == 1:
            positions.append(filter_df.loc[num, 'Position'])
            names.append(filter_df.loc[num, 'Nickname'])
            ids.append(filter_df.loc[num, 'Id'])
            salaries.append(filter_df.loc[num, 'Salary'])
            game_infos.append(filter_df.loc[num, 'Game'])
            teams.append(filter_df.loc[num, 'Team_x'])
            ppgs.append(filter_df.loc[num, 'FPPG'])
            ranks.append(filter_df.loc[num, 'PredictRank_LGBM'])
            predictions.append(filter_df.loc[num, 'Prediction_LGBM'])
            pos_C.append(filter_df.loc[num, 'Pos_C']),
            pos_W.append(filter_df.loc[num, 'Pos_W']),
            pos_D.append(filter_df.loc[num, 'Pos_D']),
            pos_G.append(filter_df.loc[num, 'Pos_G']),
            
    df = pd.DataFrame({'Position':positions,'Name':names, 'ID':ids, 'Salary':salaries, 
                       'Game Info':game_infos,'TeamAbbrev':teams,'FPPG':ppgs,
                       'Rank':ranks,'LineupNum':lineup_num, 'Prediction':predictions,
                       'Pos_C':pos_C, 'Pos_W':pos_W, 'Pos_D':pos_D, 'Pos_G':pos_G,})
    return df

In [34]:
csv_dfs = []
# function to display transformed lineups optimized by PULP
def gen_lineups(df, num_lineups, overlap, player_names = [0], nonplayer_names = [0]):
    lineup_list = []
    for i in range(num_lineups):
        lineup_list.append(pred_lineup(df, lineup_list, overlap, player_names, nonplayer_names))
    print('Number of generated lineups:', len(lineup_list))
    
    for i in range(len(lineup_list)):
        lineup_df = transform_lineup(lineup_list[i], i+1)
        lineup_df['FD_ID'] = lineup_df['ID'] + ':' + lineup_df['Name']
        
        # custom sort for lineup
        sort_dict = {'C': 0, 'W': 1, 'D': 2, 'G': 3} 
        display(lineup_df.sort_values(by=['Position'], key=lambda x: x.map(sort_dict)).iloc[:, :10])
        lineup_df.sort_values(by=['Position'], key=lambda x: x.map(sort_dict)).iloc[:, :10].to_csv('lineups/lineup_' + str(i) + '_' + '_'.join(lineup_df['TeamAbbrev'].value_counts().keys().tolist()) + '_' + curr_date + '.csv')
        print('salary sum:', lineup_df['Salary'].sum(), '\nlineup FPPG mean:', lineup_df['FPPG'].mean(), '\nlineup Rank mean:', lineup_df['Rank'].mean(), '\nlineup Predicted FP:', lineup_df['Prediction'].sum())
        
        v_counts = lineup_df['Position'].value_counts()
#         display(v_counts)
        values = lineup_df['Position'].value_counts().keys().tolist()
        counts = lineup_df['Position'].value_counts().tolist()
#         print(counts)

In [ ]:
# lineup generation (don't enter player name as 4th arg. (which should be a list) unless specific player is desired)
gen_lineups(filter_df, 5, 4, [0],[0])

In [256]:
filter_df[['Nickname','Salary', 'FPPG', 'Prediction_LGBM']]

,Nickname,Salary,FPPG,Prediction_LGBM
0,Connor McDavid,10600,25.771430,20.503430
1,Nathan MacKinnon,10300,22.730555,19.098275
2,Steven Stamkos,8900,18.514894,17.924714
3,Evgeni Malkin,7400,14.745833,16.926660
4,Ryan Nugent-Hopkins,8400,16.028572,16.465592
...,...,...,...,...
464,Anthony Stolarz,7600,8.800000,5.470033
465,Filip Gustavsson,8000,16.872728,5.269154
466,Kaapo Kahkonen,6800,7.733333,4.772807
467,Adin Hill,7400,13.790476,4.610524


## Single Game Lineups

In [227]:
import os
import pickle
import numpy as np
import pandas as pd

pred_dir = 'Predictions/'
roster_file = 'rosters/MIN_ARI.csv'

In [228]:
# read in model predictions
c_preds = pd.read_csv(pred_dir + 'C_Predictions_LGBM_'+str(curr_date)+'.csv')
w_preds = pd.read_csv(pred_dir + 'W_Predictions_LGBM_'+str(curr_date)+'.csv')
d_preds = pd.read_csv(pred_dir + 'D_Predictions_LGBM_'+str(curr_date)+'.csv')
g_preds = pd.read_csv(pred_dir + 'G_Predictions_LGBM_'+str(curr_date)+'.csv')

# concat predictions to a single dataframe
all_preds = pd.concat([c_preds, w_preds, d_preds, g_preds]).reset_index(drop = True)
all_preds = all_preds.drop(['Unnamed: 0'], axis = 1)

In [229]:
all_preds = all_preds.rename(columns = {'Name' : 'Nickname'})

# fix names with accent marks/suffixes
import unidecode

for i in range(len(all_preds)):
    # accent mark fix
    all_preds.iloc[i, 2] = unidecode.unidecode(all_preds.iloc[i, 2])
    # drop Jrs.
    if all_preds.iloc[i, 2][-3:] == 'Jr.' or all_preds.iloc[i, 2][-2:] == 'IV':
        all_preds.iloc[i, 2] = all_preds.iloc[i, 2][:-3]

# read in fanduel salaries
fd_df = pd.read_csv(roster_file)

In [230]:
# merge predictions and FD df so salaries are available
all_preds_merged = pd.merge(all_preds, fd_df, on='Nickname')

# drop players w/ 0 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] != 0)].reset_index(drop = True)

# keep players w/ > 2.1 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] >= 2.1)].reset_index(drop = True)

# drop injured players
all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'O')].reset_index(drop = True)
all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'IR')].reset_index(drop = True)
# all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'NA')].reset_index(drop = True)
# all_preds_merged = all_preds_merged[(all_preds_merged['Injury Indicator'] != 'GTD')].reset_index(drop = True)

# drop players based on how many games they've played
# makes sure that players who don't play aren't considered
all_preds_merged = all_preds_merged[(all_preds_merged.Played >= all_preds_merged.Played.quantile(.2))].reset_index(drop = True)

# drop players based on salary cutoffs
# makes sure that bad players are excluded for each position
all_preds_merged = all_preds_merged[(all_preds_merged.Salary >= 6500)].reset_index(drop = True)

# drop unnamed columns originating from fd_df
cols = all_preds_merged.columns
for col in cols:
    if 'Unnamed'in col:
        all_preds_merged.drop(col, inplace = True, axis = 1)

In [231]:
all_preds_merged

,Season,Date,Nickname,Team_x,Opponent_x,Prediction_LGBM,PredictRank_LGBM,Id,Position,First Name,...,FPPG,Played,Salary,Game,Team_y,Opponent_y,Injury Indicator,Injury Details,Tier,Roster Position
0,2022,20230206,Joel Eriksson Ek,MIN,ARI,12.751383,16.0,86682-73949,C,Joel,...,14.458333,48.0,13000,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
1,2022,20230206,Nick Schmaltz,ARI,MIN,9.187652,28.0,86682-73889,W,Nick,...,10.668421,38.0,10000,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
2,2022,20230206,Barrett Hayton,ARI,MIN,8.365028,34.0,86682-97108,C,Barrett,...,7.000000,50.0,7500,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
3,2022,20230206,Frederick Gaudreau,MIN,ARI,7.948367,36.0,86682-59209,W,Frederick,...,8.618750,48.0,9500,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
4,2022,20230206,Nick Bjugstad,ARI,MIN,7.867089,39.0,86682-39249,C,Nick,...,7.864000,50.0,9000,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
5,2022,20230206,Sam Steel,MIN,ARI,7.143985,43.0,86682-70928,C,Sam,...,6.931250,48.0,7500,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
6,2022,20230206,Travis Boyd,ARI,MIN,6.465743,50.0,86682-62275,C,Travis,...,6.194000,50.0,7000,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
7,2022,20230206,Connor Dewar,MIN,ARI,5.382044,62.0,86682-97056,W,Connor,...,5.463830,47.0,7000,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
8,2022,20230206,Jack McBain,ARI,MIN,5.360837,63.0,86682-167209,C,Jack,...,5.024000,50.0,6500,MIN@ARI,ARI,MIN,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL
9,2022,20230206,Mason Shaw,MIN,ARI,2.630591,195.0,86682-82122,C,Mason,...,6.517647,34.0,7500,MIN@ARI,MIN,ARI,NaN,NaN,NaN,Captain - 1.5x Pts/UTIL


In [232]:
# players by position
all_preds_merged['Position'].value_counts()

W    13
D    12
C     8
Name: Position, dtype: int64

In [233]:
# filter_df for pulp
filter_df = all_preds_merged.copy()
filter_df['MaxUsed'] = 0
filter_df['Pos_C'] = np.where(filter_df['Position']=='C', 1, 0)
filter_df['Pos_W'] = np.where(filter_df['Position']=='W', 1, 0)
filter_df['Pos_D'] = np.where(filter_df['Position']=='D', 1, 0)
filter_df['Pos_G'] = np.where(filter_df['Position']=='G', 1, 0)
filter_df['Pos_All'] = 1

In [234]:
# ensure players are from different teams (at least 1 player from each team)
teams = filter_df['Team_x'].value_counts().index.tolist()
filter_df['Team_1'] = np.where((filter_df['Team_x']==teams[0]), 1, 0)
filter_df['Team_2'] = np.where((filter_df['Team_x']==teams[1]), 1, 0)

In [235]:
encoded = pd.get_dummies(filter_df['Nickname']) #<-- One-Hot Encoding 

In [236]:
filter_df = filter_df.join(encoded) #<-- joining it to the raw_data table

In [237]:
encoded.columns.tolist()

['Alex Goligoski',
 'Barrett Hayton',
 'Brandon Duhaime',
 'Calen Addison',
 'Christian Fischer',
 'Clayton Keller',
 'Connor Dewar',
 'Dylan Guenther',
 'Frederick Gaudreau',
 'J.J. Moser',
 'Jack McBain',
 'Jakob Chychrun',
 'Jared Spurgeon',
 'Joel Eriksson Ek',
 'Jon Merrill',
 'Jonas Brodin',
 'Josh Brown',
 'Juuso Valimaki',
 'Kirill Kaprizov',
 'Lawson Crouse',
 'Marcus Foligno',
 'Mason Shaw',
 'Matias Maccelli',
 'Mats Zuccarello',
 'Matt Dumba',
 'Nick Bjugstad',
 'Nick Ritchie',
 'Nick Schmaltz',
 'Patrik Nemeth',
 'Ryan Hartman',
 'Sam Steel',
 'Travis Boyd',
 'Troy Stecher']

In [238]:
import pulp

def pred_lineup_singlegame(df, lineups, overlap, player_names = [0], nonplayer_names = [0]):
    #Initialize our PuLP problem, we choose pulp.LpMinimize because we are going to want the lowest ranked players numerically. (1st is better than 2nd, technically smaller though)
    prob = pulp.LpProblem('NHL_DK', pulp.LpMaximize)
    
    #Count How Many Different Players We Have (Including Team D)
    num_all = df.shape[0]
    
    #Create Salary Cap Variable
    salary_cap = 55000
    
    #Create List Of PuLP player variables
    players_lineup = [pulp.LpVariable("Player_{}".format(i+1), cat="Binary") for i in range(num_all)]
    
    #Tell our PuLP solver that we want our lineup to have our prediction field to be maximized
    prob += pulp.lpSum((pulp.lpSum(df.loc[i, 'Prediction_LGBM'] * players_lineup[i] for i in range(num_all))))
    
    # Make sure only no goalies are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_G'] * players_lineup[i] for i in range(num_all)))  == 0)
    
    #Make Sure To Select 9 Players Total
    prob += ((pulp.lpSum(df.loc[i, 'Pos_All'] * players_lineup[i] for i in range(num_all))) == 5)
    
    #add the salary constraint
    prob += ((pulp.lpSum(df.loc[i, 'Salary'] * players_lineup[i] for i in range(num_all))) <= salary_cap)
    
    #Add Our MaxUsed Constraint
    prob += ((pulp.lpSum(df.loc[i, 'MaxUsed'] * players_lineup[i] for i in range(num_all))) == 0)
    
    #Add constraints for teams so at least 1 player from each team is selected
    prob += ((pulp.lpSum(df.loc[i, 'Team_1'] * players_lineup[i] for i in range(num_all))) <= 4)
    prob += ((pulp.lpSum(df.loc[i, 'Team_2'] * players_lineup[i] for i in range(num_all))) <= 4)
    
    #select specific player, if indicated
    if player_names != [0]:
        for player_name in player_names:
            prob += ((pulp.lpSum(df.loc[i, player_name] * players_lineup[i] for i in range(num_all)))  == 1)
            
    #remove specific player from consideration, if indicated
    if nonplayer_names != [0]:
        for nonplayer_name in nonplayer_names:
            prob += ((pulp.lpSum(df.loc[i, nonplayer_name] * players_lineup[i] for i in range(num_all)))  == 0)
    
    # add constraint to select only 1 of each player
    for player in encoded.columns.tolist():
        prob += ((pulp.lpSum(df.loc[i, player] * players_lineup[i] for i in range(num_all))) <= 1)
    
    #variance constraints - each lineup can't have more than the num overlap of any combination of players in any previous lineups
    for i in range(len(lineups)):
        prob += ((pulp.lpSum(lineups[i][k] * players_lineup[k] for k in range(num_all))) <= overlap)
        
    #solve the problem
    status = prob.solve()
    
    #check if the optimizer found an optimal solution
    if status != pulp.LpStatusOptimal:
        print('Only {} feasible lineups produced'.format(len(lineups)), '\n')
        return None
    lineup_copy = []
    for i in range(num_all):
        if players_lineup[i].varValue >= 0.9 and players_lineup[i].varValue <= 1.1:
            lineup_copy.append(1)
        else:
            lineup_copy.append(0)

    return lineup_copy

In [239]:
# function to tranform lineups resulting from PULP optimization
def transform_lineup(lineup, lineup_num):
    positions, names, ids, salaries = [], [], [], []
    game_infos, teams, ppgs, ranks = [], [], [], []
    predictions = []
    pos_C = []
    pos_W = []
    pos_D = []
    pos_G = []

    players_lineup = lineup[:len(lineup)]
    for num, player in enumerate(players_lineup):
        if player == 1:
            positions.append(filter_df.loc[num, 'Position'])
            names.append(filter_df.loc[num, 'Nickname'])
            ids.append(filter_df.loc[num, 'Id'])
            salaries.append(filter_df.loc[num, 'Salary'])
            game_infos.append(filter_df.loc[num, 'Game'])
            teams.append(filter_df.loc[num, 'Team_x'])
            ppgs.append(filter_df.loc[num, 'FPPG'])
            ranks.append(filter_df.loc[num, 'PredictRank_LGBM'])
            predictions.append(filter_df.loc[num, 'Prediction_LGBM'])
            pos_C.append(filter_df.loc[num, 'Pos_C']),
            pos_W.append(filter_df.loc[num, 'Pos_W']),
            pos_D.append(filter_df.loc[num, 'Pos_D']),
            pos_G.append(filter_df.loc[num, 'Pos_G']),
            
    df = pd.DataFrame({'Position':positions,'Name':names, 'ID':ids, 'Salary':salaries, 
                       'Game Info':game_infos,'TeamAbbrev':teams,'FPPG':ppgs,
                       'Rank':ranks,'LineupNum':lineup_num, 'Prediction':predictions,
                       'Pos_C':pos_C, 'Pos_W':pos_W, 'Pos_D':pos_D, 'Pos_G':pos_G,})
    return df

In [240]:
csv_dfs = []
# function to display transformed lineups optimized by PULP
def gen_lineups_singlegame(df, num_lineups, overlap, player_names = [0], nonplayer_names = [0]):
    lineup_list = []
    for i in range(num_lineups):
        lineup_list.append(pred_lineup_singlegame(df, lineup_list, overlap, player_names, nonplayer_names))
    print('Number of generated lineups:', len(lineup_list))
    
    for i in range(len(lineup_list)):
        lineup_df = transform_lineup(lineup_list[i], i+1)
        lineup_df['FD_ID'] = lineup_df['ID'] + ':' + lineup_df['Name']
        
        # custom sort for lineup
        sort_dict = {'C': 0, 'W': 1, 'D': 2, 'G': 3} 
        lineup_df = lineup_df.sort_values(by=['Prediction'], ascending = False)
        display(lineup_df[['Name', 'Salary', 'FPPG', 'TeamAbbrev', 'Prediction']])
        lineup_df.to_csv('lineups/lineup_' + str(i) + '_' + '_'.join(lineup_df['TeamAbbrev'].value_counts().keys().tolist()) + '_' + curr_date + '.csv')
        print('salary sum:', lineup_df['Salary'].sum(), '\nlineup FPPG mean:', lineup_df['FPPG'].mean(), '\nlineup Rank mean:', lineup_df['Rank'].mean(), '\nlineup Predicted FP:', lineup_df['Prediction'].sum())
        v_counts = lineup_df['Position'].value_counts()
        values = lineup_df['Position'].value_counts().keys().tolist()
        counts = lineup_df['Position'].value_counts().tolist()

In [241]:
# lineup generation (don't enter player name as 4th arg. (which should be a list) unless specific player is desired)
gen_lineups_singlegame(filter_df, 5, 3, [0],[0])

Number of generated lineups: 5


,Name,Salary,FPPG,TeamAbbrev,Prediction
2,Kirill Kaprizov,16000,18.997917,MIN,17.185094
4,Jakob Chychrun,13500,14.135294,ARI,15.016674
3,Clayton Keller,11500,12.430000,ARI,11.788541
0,Barrett Hayton,7500,7.000000,ARI,8.365028
1,Jack McBain,6500,5.024000,ARI,5.360837


salary sum: 55000 
lineup FPPG mean: 11.517442282284007 
lineup Rank mean: 23.8 
lineup Predicted FP: 57.71617309654427


,Name,Salary,FPPG,TeamAbbrev,Prediction
3,Kirill Kaprizov,16000,18.997917,MIN,17.185094
4,Jakob Chychrun,13500,14.135294,ARI,15.016674
0,Nick Schmaltz,10000,10.668421,ARI,9.187652
1,Barrett Hayton,7500,7.000000,ARI,8.365028
2,Sam Steel,7500,6.931250,MIN,7.143985


salary sum: 54500 
lineup FPPG mean: 11.546576523756219 
lineup Rank mean: 22.4 
lineup Predicted FP: 56.89843211090286


...

salary sum: 55000 
lineup FPPG mean: 11.616692292456534 
lineup Rank mean: 20.6 
lineup Predicted FP: 56.81597716795072


,Name,Salary,FPPG,TeamAbbrev,Prediction
3,Kirill Kaprizov,16000,18.997917,MIN,17.185094
0,Joel Eriksson Ek,13000,14.458333,MIN,12.751383
4,Clayton Keller,11500,12.430000,ARI,11.788541
1,Barrett Hayton,7500,7.000000,ARI,8.365028
2,Travis Boyd,7000,6.194000,ARI,6.465743


salary sum: 55000 
lineup FPPG mean: 11.816050150553385 
lineup Rank mean: 23.8 
lineup Predicted FP: 56.55578781429165


,Name,Salary,FPPG,TeamAbbrev,Prediction
1,Kirill Kaprizov,16000,18.997917,MIN,17.185094
3,Jakob Chychrun,13500,14.135294,ARI,15.016674
2,Clayton Keller,11500,12.430000,ARI,11.788541
0,Sam Steel,7500,6.931250,MIN,7.143985
4,Troy Stecher,6500,4.416000,ARI,5.287913


salary sum: 55000 
lineup FPPG mean: 11.3820923575607 
lineup Rank mean: 25.2 
lineup Predicted FP: 56.42220593772659


In [242]:
filter_df.sort_values(by='Prediction_LGBM', ascending = False)

,Season,Date,Nickname,Team_x,Opponent_x,Prediction_LGBM,PredictRank_LGBM,Id,Position,First Name,...,Mats Zuccarello,Matt Dumba,Nick Bjugstad,Nick Ritchie,Nick Schmaltz,Patrik Nemeth,Ryan Hartman,Sam Steel,Travis Boyd,Troy Stecher
10,2022,20230206,Kirill Kaprizov,MIN,ARI,17.185094,4.0,86682-135875,W,Kirill,...,0,0,0,0,0,0,0,0,0,0
21,2022,20230206,Jakob Chychrun,ARI,MIN,15.016674,3.0,86682-70914,D,Jakob,...,0,0,0,0,0,0,0,0,0,0
11,2022,20230206,Mats Zuccarello,MIN,ARI,13.441216,9.0,86682-12253,W,Mats,...,1,0,0,0,0,0,0,0,0,0
0,2022,20230206,Joel Eriksson Ek,MIN,ARI,12.751383,16.0,86682-73949,C,Joel,...,0,0,0,0,0,0,0,0,0,0
12,2022,20230206,Clayton Keller,ARI,MIN,11.788541,15.0,86682-70905,W,Clayton,...,0,0,0,0,0,0,0,0,0,0
22,2022,20230206,Jared Spurgeon,MIN,ARI,9.783439,12.0,86682-12812,D,Jared,...,0,0,0,0,0,0,0,0,0,0
1,2022,20230206,Nick Schmaltz,ARI,MIN,9.187652,28.0,86682-73889,W,Nick,...,0,0,0,0,1,0,0,0,0,0
2,2022,20230206,Barrett Hayton,ARI,MIN,8.365028,34.0,86682-97108,C,Barrett,...,0,0,0,0,0,0,0,0,0,0
13,2022,20230206,Ryan Hartman,MIN,ARI,8.276134,33.0,86682-43176,C,Ryan,...,0,0,0,0,0,0,1,0,0,0
3,2022,20230206,Frederick Gaudreau,MIN,ARI,7.948367,36.0,86682-59209,W,Frederick,...,0,0,0,0,0,0,0,0,0,0


## Slate Lineups (Yahoo)

In [260]:
import os
import pickle
import numpy as np
import pandas as pd

pred_dir = 'Predictions/'
roster_file = 'rosters/yahoo/yahoo_6gameslate.csv'

In [261]:
# read in model predictions
c_preds = pd.read_csv(pred_dir + 'C_Predictions_LGBM_'+str(curr_date)+'.csv')
w_preds = pd.read_csv(pred_dir + 'W_Predictions_LGBM_'+str(curr_date)+'.csv')
d_preds = pd.read_csv(pred_dir + 'D_Predictions_LGBM_'+str(curr_date)+'.csv')
g_preds = pd.read_csv(pred_dir + 'G_Predictions_LGBM_'+str(curr_date)+'.csv')

# concat predictions to a single dataframe
all_preds = pd.concat([c_preds, w_preds, d_preds, g_preds]).reset_index(drop = True)
all_preds = all_preds.drop(['Unnamed: 0'], axis = 1)

In [262]:
all_preds = all_preds.rename(columns = {'Name' : 'Nickname'})

# fix names with accent marks/suffixes
import unidecode

for i in range(len(all_preds)):
    # accent mark fix
    all_preds.iloc[i, 2] = unidecode.unidecode(all_preds.iloc[i, 2])
    # drop Jrs.
    if all_preds.iloc[i, 2][-3:] == 'Jr.' or all_preds.iloc[i, 2][-2:] == 'IV':
        all_preds.iloc[i, 2] = all_preds.iloc[i, 2][:-3]

# read in fanduel salaries
fd_df = pd.read_csv(roster_file)
fd_df['Nickname'] = fd_df[['First Name', 'Last Name']].apply(lambda x: ' '.join(x), axis=1)

In [263]:
for i in range(len(fd_df)):
    # accent mark fix
    fd_df.iloc[i, 13] = unidecode.unidecode(fd_df.iloc[i, 13])
    # drop Jrs., Srs/
    if fd_df.iloc[i, 13][-3:] == 'Jr.' or fd_df.iloc[i, 13][-3:] == 'Sr.':
        fd_df.iloc[i, 13] = fd_df.iloc[i, 13][:-4]
    elif fd_df.iloc[i, 13][-2:] == 'IV':
        fd_df.iloc[i, 13] = fd_df.iloc[i, 13][:-3]

# merge predictions and FD df so salaries are available
all_preds_merged = pd.merge(all_preds, fd_df, on='Nickname')

In [264]:
# merge predictions and FD df so salaries are available
all_preds_merged = pd.merge(all_preds, fd_df, on='Nickname')

# drop players w/ 0 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] != 0)].reset_index(drop = True)

# keep players w/ > 2.1 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] >= 2.1)].reset_index(drop = True)

# drop injured players
all_preds_merged = all_preds_merged[(all_preds_merged['Injury Status'] != 'O')].reset_index(drop = True)
all_preds_merged = all_preds_merged[(all_preds_merged['Injury Status'] != 'IR')].reset_index(drop = True)

# drop players based on salary cutoffs
# makes sure that bad players are excluded for each position
all_preds_merged = all_preds_merged[(all_preds_merged.Salary >= 11)].reset_index(drop = True)

# drop unnamed columns originating from fd_df
cols = all_preds_merged.columns
for col in cols:
    if 'Unnamed'in col:
        all_preds_merged.drop(col, inplace = True, axis = 1)

In [265]:
all_preds_merged

,Season,Date,Nickname,Team_x,Opponent_x,Prediction_LGBM,PredictRank_LGBM,ID,First Name,Last Name,ID + Name,Position,Team_y,Opponent_y,Game,Time,Salary,FPPG,Injury Status,Starting
0,2022,20230206,Steven Stamkos,TB,FLA,17.090487,1.0,nhl.p.4471$nhl.g.2466453,Steven,Stamkos,nhl.p.4471$nhl.g.2466453 - Steven Stamkos,C,TB,FLA,TB@FLA,7:00PM EST,31,10.8,,
1,2022,20230206,Mika Zibanejad,NYR,CGY,16.947355,2.0,nhl.p.5367$nhl.g.2466460,Mika,Zibanejad,nhl.p.5367$nhl.g.2466460 - Mika Zibanejad,C,NYR,CGY,CGY@NYR,7:30PM EST,29,10.0,,
2,2022,20230206,Nico Hischier,NJ,VAN,16.141115,3.0,nhl.p.7516$nhl.g.2466455,Nico,Hischier,nhl.p.7516$nhl.g.2466455 - Nico Hischier,C,NJ,VAN,VAN@NJ,7:30PM EST,23,9.5,,
3,2022,20230206,Aleksander Barkov,FLA,TB,15.725858,4.0,nhl.p.5981$nhl.g.2466453,Aleksander,Barkov,nhl.p.5981$nhl.g.2466453 - Aleksander Barkov,C,FLA,TB,TB@FLA,7:00PM EST,28,9.8,,
4,2022,20230206,Carter Verhaeghe,FLA,TB,14.767676,5.0,nhl.p.6062$nhl.g.2466453,Carter,Verhaeghe,nhl.p.6062$nhl.g.2466453 - Carter Verhaeghe,LW,FLA,TB,TB@FLA,7:00PM EST,20,8.2,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2022,20230206,Collin Delia,VAN,NJD,5.627325,29.0,nhl.p.7737$nhl.g.2466455,Collin,Delia,nhl.p.7737$nhl.g.2466455 - Collin Delia,G,VAN,NJ,VAN@NJ,7:30PM EST,21,8.6,,Yes
147,2022,20230206,Brian Elliott,TB,FLA,5.173397,34.0,nhl.p.4136$nhl.g.2466453,Brian,Elliott,nhl.p.4136$nhl.g.2466453 - Brian Elliott,G,TB,FLA,TB@FLA,7:00PM EST,26,11.6,,No
148,2022,20230206,Anthony Stolarz,ANA,DAL,4.761114,39.0,nhl.p.5734$nhl.g.2466462,Anthony,Stolarz,nhl.p.5734$nhl.g.2466462 - Anthony Stolarz,G,ANA,DAL,ANA@DAL,8:30PM EST,25,6.8,,No
149,2022,20230206,Alex Lyon,FLA,TB,3.960710,53.0,nhl.p.7089$nhl.g.2466453,Alex,Lyon,nhl.p.7089$nhl.g.2466453 - Alex Lyon,G,FLA,TB,TB@FLA,7:00PM EST,20,9.0,,No


In [266]:
# players by position
all_preds_merged['Position'].value_counts()

D     43
C     36
LW    26
G     24
RW    22
Name: Position, dtype: int64

In [267]:
# filter_df for pulp
filter_df = all_preds_merged.copy()
filter_df['MaxUsed'] = 0
filter_df['Pos_C'] = np.where(filter_df['Position']=='C', 1, 0)
filter_df['Pos_W'] = np.where((filter_df['Position']=='LW') | (filter_df['Position']=='RW'), 1, 0)
filter_df['Pos_D'] = np.where(filter_df['Position']=='D', 1, 0)
filter_df['Pos_G'] = np.where(filter_df['Position']=='G', 1, 0)
filter_df['Starting_G'] = np.where((filter_df['Position']=='G') & (filter_df['Starting'] == 'Yes'), 1, 0)
filter_df['Pos_All'] = 1

In [268]:
encoded = pd.get_dummies(filter_df['Nickname']) #<-- One-Hot Encoding 

In [269]:
filter_df = filter_df.join(encoded) #<-- joining it to the raw_data table

In [270]:
encoded.columns.tolist()

['Aaron Ekblad',
 'Adam Fox',
 'Adam Henrique',
 'Adam Pelech',
 'Aleksander Barkov',
 'Alex Killorn',
 'Alex Lyon',
 'Alexis Lafreniere',
 'Anders Lee',
 'Andrei Kuzmenko',
 'Andrei Vasilevskiy',
 'Andrew Mangiapane',
 'Anthony Cirelli',
 'Anthony Stolarz',
 'Anton Lundell',
 'Artemi Panarin',
 'Barclay Goodrow',
 'Barrett Hayton',
 'Blake Coleman',
 'Bo Horvat',
 'Braden Schneider',
 'Brandon Hagel',
 'Brandon Montour',
 'Brayden Point',
 'Brian Elliott',
 'Brock Boeser',
 'Brock Nelson',
 'Cam Fowler',
 'Cam York',
 'Carter Hart',
 'Carter Verhaeghe',
 'Chris Kreider',
 'Clayton Keller',
 'Colin Miller',
 'Collin Delia',
 'Conor Garland',
 'Corey Perry',
 'Damon Severson',
 'Daniel Vladar',
 'Dawson Mercer',
 'Dillon Dube',
 'Dougie Hamilton',
 'Elias Lindholm',
 'Elias Pettersson',
 'Erik Haula',
 'Esa Lindell',
 'Felix Sandstrom',
 'Filip Chytil',
 'Filip Gustavsson',
 'Frank Vatrano',
 'Frederick Gaudreau',
 'Gustav Forsling',
 'Igor Shesterkin',
 'Ilya Sorokin',
 'Ivan Provorov'

In [271]:
filter_df.columns.tolist()[:27]

['Season',
 'Date',
 'Nickname',
 'Team_x',
 'Opponent_x',
 'Prediction_LGBM',
 'PredictRank_LGBM',
 'ID',
 'First Name',
 'Last Name',
 'ID + Name',
 'Position',
 'Team_y',
 'Opponent_y',
 'Game',
 'Time',
 'Salary',
 'FPPG',
 'Injury Status',
 'Starting',
 'MaxUsed',
 'Pos_C',
 'Pos_W',
 'Pos_D',
 'Pos_G',
 'Starting_G',
 'Pos_All']

In [272]:
import pulp

def pred_lineup_yahoo(df, lineups, overlap, salary, player_names = [0], nonplayer_names = [0]):
    #Initialize our PuLP problem, we choose pulp.LpMinimize because we are going to want the lowest ranked players numerically. (1st is better than 2nd, technically smaller though)
    prob = pulp.LpProblem('NHL_Yahoo', pulp.LpMaximize)
    
    #Count How Many Different Players We Have (Including Team D)
    num_all = df.shape[0]
    
    # get unique teams
    uniq_teams = df['Team_x'].unique().tolist()
    
    # one-hot encoding for teams
    team_ohe = pd.get_dummies(filter_df['Team_x'])
    df = df.join(team_ohe)
    
    #Create Salary Cap Variable
    salary_cap = salary
    
    #Create List Of PuLP player variables
    players_lineup = [pulp.LpVariable("Player_{}".format(i+1), cat="Binary") for i in range(num_all)]
    
    #Tell our PuLP solver that we want our lineup to have our prediction field to be maximized
    prob += pulp.lpSum((pulp.lpSum(df.loc[i, 'Prediction_LGBM'] * players_lineup[i] for i in range(num_all))))
    
    # Make sure only 2 Gs are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_G'] * players_lineup[i] for i in range(num_all)))  == 2)
    
    # Make sure goalies are starting
    prob += ((pulp.lpSum(df.loc[i, 'Starting_G'] * players_lineup[i] for i in range(num_all)))  == 2)
    
    # Make sure only 2 Cs are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_C'] * players_lineup[i] for i in range(num_all)))  == 2)
    
    # Make sure only 3 Ws are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_W'] * players_lineup[i] for i in range(num_all)))  == 3)
    
    # Make sure only 2 Ds are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_D'] * players_lineup[i] for i in range(num_all)))  == 2)
    
    #Make Sure To Select 9 Players Total
    prob += ((pulp.lpSum(df.loc[i, 'Pos_All'] * players_lineup[i] for i in range(num_all))) == 9)
    
    #add the salary constraint
    prob += ((pulp.lpSum(df.loc[i, 'Salary'] * players_lineup[i] for i in range(num_all))) <= salary_cap)
    
    #Add Our MaxUsed Constraint
    prob += ((pulp.lpSum(df.loc[i, 'MaxUsed'] * players_lineup[i] for i in range(num_all))) == 0)
    
    # constraint to not select more than 4 players from a team
    for team in uniq_teams:
        prob += ((pulp.lpSum(df.loc[i, team] * players_lineup[i] for i in range(num_all)))  <= 4)
    
    #select specific player, if indicated
    if player_names != [0]:
        for player_name in player_names:
            prob += ((pulp.lpSum(df.loc[i, player_name] * players_lineup[i] for i in range(num_all)))  == 1)
            
    #remove specific player from consideration, if indicated
    if nonplayer_names != [0]:
        for nonplayer_name in nonplayer_names:
            prob += ((pulp.lpSum(df.loc[i, nonplayer_name] * players_lineup[i] for i in range(num_all)))  == 0)
    
    # add constraint to select only 1 of each player
    for player in encoded.columns.tolist():
        prob += ((pulp.lpSum(df.loc[i, player] * players_lineup[i] for i in range(num_all))) <= 1)
    
    #variance constraints - each lineup can't have more than the num overlap of any combination of players in any previous lineups
    for i in range(len(lineups)):
        prob += ((pulp.lpSum(lineups[i][k] * players_lineup[k] for k in range(num_all))) <= overlap)
        
    #solve the problem
    status = prob.solve()
    
    #check if the optimizer found an optimal solution
    if status != pulp.LpStatusOptimal:
        print('Only {} feasible lineups produced'.format(len(lineups)), '\n')
        return None
    lineup_copy = []
    
    for i in range(num_all):
        if players_lineup[i].varValue >= 0.9 and players_lineup[i].varValue <= 1.1:
            lineup_copy.append(1)
        else:
            lineup_copy.append(0)

    return lineup_copy

In [273]:
# function to tranform lineups resulting from PULP optimization
def transform_lineup_yahoo(lineup, lineup_num):
    positions, names, ids, salaries = [], [], [], []
    game_infos, teams, ppgs, ranks = [], [], [], []
    predictions = []
    pos_C = []
    pos_W = []
    pos_D = []
    pos_G = []

    players_lineup = lineup[:len(lineup)]
    for num, player in enumerate(players_lineup):
        if player == 1:
            positions.append(filter_df.loc[num, 'Position'])
            names.append(filter_df.loc[num, 'Nickname'])
            ids.append(filter_df.loc[num, 'ID'])
            salaries.append(filter_df.loc[num, 'Salary'])
            game_infos.append(filter_df.loc[num, 'Game'])
            teams.append(filter_df.loc[num, 'Team_x'])
            ppgs.append(filter_df.loc[num, 'FPPG'])
            ranks.append(filter_df.loc[num, 'PredictRank_LGBM'])
            predictions.append(filter_df.loc[num, 'Prediction_LGBM'])
            pos_C.append(filter_df.loc[num, 'Pos_C']),
            pos_W.append(filter_df.loc[num, 'Pos_W']),
            pos_D.append(filter_df.loc[num, 'Pos_D']),
            pos_G.append(filter_df.loc[num, 'Pos_G']),
            
    df = pd.DataFrame({'Position':positions,'Name':names, 'Salary':salaries, 
                       'Game Info':game_infos,'TeamAbbrev':teams,'FPPG':ppgs,
                       'Rank':ranks,'LineupNum':lineup_num, 'Prediction':predictions,
                       'Pos_C':pos_C, 'Pos_W':pos_W, 'Pos_D':pos_D, 'Pos_G':pos_G,})
    return df

In [274]:
csv_dfs = []
# function to display transformed lineups optimized by PULP
def gen_lineups_yahoo(df, num_lineups, overlap, salary, player_names = [0], nonplayer_names = [0]):
    lineup_list = []
    for i in range(num_lineups):
        lineup_list.append(pred_lineup_yahoo(df, lineup_list, overlap, salary, player_names, nonplayer_names))
    print('Number of generated lineups:', len(lineup_list))
    
    for i in range(len(lineup_list)):
        lineup_df = transform_lineup_yahoo(lineup_list[i], i+1)
        
        # custom sort for lineup
        sort_dict = {'G': 0, 'C': 1, 'LW': 2, 'RW': 3, 'D': 4} 
        display(lineup_df.sort_values(by=['Position'], key=lambda x: x.map(sort_dict)).iloc[:, :9])
        lineup_df.sort_values(by=['Position'], key=lambda x: x.map(sort_dict)).iloc[:, :9].to_csv('lineups/yahoo/lineup_' + str(i) + '_' + '_'.join(lineup_df['TeamAbbrev'].value_counts().keys().tolist()) + '_' + curr_date + '.csv')
        print('salary sum:', lineup_df['Salary'].sum(), '\nlineup FPPG mean:', lineup_df['FPPG'].mean(), '\nlineup Rank mean:', lineup_df['Rank'].mean(), '\nlineup Predicted FP:', lineup_df['Prediction'].sum())

In [275]:
# lineup generation (don't enter player name as 4th arg. (which should be a list) unless specific player is desired)
gen_lineups_yahoo(filter_df, 5, 4, 200, [0],[0])

Number of generated lineups: 5


,Position,Name,Salary,Game Info,TeamAbbrev,FPPG,Rank,LineupNum,Prediction
7,G,Karel Vejmelka,25,MIN@ARI,ARI,11.3,6.0,1,14.191132
8,G,John Gibson,21,ANA@DAL,ANA,9.9,7.0,1,14.100984
1,C,Kevin Hayes,17,NYI@PHI,PHI,7.4,8.0,1,13.697418
2,C,Brock Nelson,17,NYI@PHI,NYI,7.8,10.0,1,13.409902
4,LW,Artemi Panarin,24,CGY@NYR,NYR,7.7,3.0,1,18.167482
0,RW,Sam Reinhart,18,TB@FLA,FLA,6.7,7.0,1,14.065486
3,RW,Jason Robertson,32,ANA@DAL,DAL,12.1,1.0,1,20.375277
5,D,Adam Fox,25,CGY@NYR,NYR,9.4,2.0,1,16.915779
6,D,Jakob Chychrun,21,MIN@ARI,ARI,8.4,3.0,1,15.016674


salary sum: 200 
lineup FPPG mean: 8.966666666666667 
lineup Rank mean: 5.222222222222222 
lineup Predicted FP: 139.94013429122654


,Position,Name,Salary,Game Info,TeamAbbrev,FPPG,Rank,LineupNum,Prediction
7,G,Marc-Andre Fleury,31,MIN@ARI,MIN,10.2,1.0,2,16.860230
8,G,John Gibson,21,ANA@DAL,ANA,9.9,7.0,2,14.100984
0,C,Nico Hischier,23,VAN@NJ,NJ,9.5,3.0,2,16.141115
2,C,Vincent Trocheck,18,CGY@NYR,NYR,6.7,14.0,2,12.966296
1,LW,Carter Verhaeghe,20,TB@FLA,FLA,8.2,5.0,2,14.767676
3,LW,Artemi Panarin,24,CGY@NYR,NYR,7.7,3.0,2,18.167482
4,RW,Frank Vatrano,17,ANA@DAL,ANA,5.1,11.0,2,12.829823
5,D,Adam Fox,25,CGY@NYR,NYR,9.4,2.0,2,16.915779
6,D,Jakob Chychrun,21,MIN@ARI,ARI,8.4,3.0,2,15.016674


salary sum: 200 
lineup FPPG mean: 8.344444444444445 
lineup Rank mean: 5.444444444444445 
lineup Predicted FP: 137.7660595929665


...

salary sum: 200 
lineup FPPG mean: 8.61111111111111 
lineup Rank mean: 4.777777777777778 
lineup Predicted FP: 136.513986148715


,Position,Name,Salary,Game Info,TeamAbbrev,FPPG,Rank,LineupNum,Prediction
7,G,Carter Hart,30,NYI@PHI,PHI,10.8,4.0,4,15.511690
8,G,John Gibson,21,ANA@DAL,ANA,9.9,7.0,4,14.100984
0,C,Nico Hischier,23,VAN@NJ,NJ,9.5,3.0,4,16.141115
2,C,Kevin Hayes,17,NYI@PHI,PHI,7.4,8.0,4,13.697418
1,LW,Carter Verhaeghe,20,TB@FLA,FLA,8.2,5.0,4,14.767676
4,LW,Artemi Panarin,24,CGY@NYR,NYR,7.7,3.0,4,18.167482
3,RW,Jason Robertson,32,ANA@DAL,DAL,12.1,1.0,4,20.375277
5,D,Tony DeAngelo,17,NYI@PHI,PHI,6.4,7.0,4,11.891721
6,D,Gustav Forsling,16,TB@FLA,FLA,6.7,8.0,4,11.268586


salary sum: 200 
lineup FPPG mean: 8.744444444444447 
lineup Rank mean: 5.111111111111111 
lineup Predicted FP: 135.921950125331


,Position,Name,Salary,Game Info,TeamAbbrev,FPPG,Rank,LineupNum,Prediction
7,G,Marc-Andre Fleury,31,MIN@ARI,MIN,10.2,1.0,5,16.860230
8,G,John Gibson,21,ANA@DAL,ANA,9.9,7.0,5,14.100984
0,C,Brock Nelson,17,NYI@PHI,NYI,7.8,10.0,5,13.409902
1,C,Vincent Trocheck,18,CGY@NYR,NYR,6.7,14.0,5,12.966296
3,LW,Artemi Panarin,24,CGY@NYR,NYR,7.7,3.0,5,18.167482
4,LW,Tomas Tatar,12,VAN@NJ,NJ,5.9,22.0,5,10.026342
2,RW,Jason Robertson,32,ANA@DAL,DAL,12.1,1.0,5,20.375277
5,D,Dougie Hamilton,28,VAN@NJ,NJ,10.0,1.0,5,17.477100
6,D,Tony DeAngelo,17,NYI@PHI,PHI,6.4,7.0,5,11.891721


salary sum: 200 
lineup FPPG mean: 8.522222222222222 
lineup Rank mean: 7.333333333333333 
lineup Predicted FP: 135.27533411717894


In [276]:
filter_df[['Nickname', 'Position','Salary', 'FPPG', 'Prediction_LGBM']]

,Nickname,Position,Salary,FPPG,Prediction_LGBM
0,Steven Stamkos,C,31,10.8,17.090487
1,Mika Zibanejad,C,29,10.0,16.947355
2,Nico Hischier,C,23,9.5,16.141115
3,Aleksander Barkov,C,28,9.8,15.725858
4,Carter Verhaeghe,LW,20,8.2,14.767676
...,...,...,...,...,...
146,Collin Delia,G,21,8.6,5.627325
147,Brian Elliott,G,26,11.6,5.173397
148,Anthony Stolarz,G,25,6.8,4.761114
149,Alex Lyon,G,20,9.0,3.960710


## Single Game Lineups (DraftKings)

In [306]:
import os
import pickle
import numpy as np
import pandas as pd

pred_dir = 'Predictions/'
roster_file = 'rosters/DK/LA_CAR.csv'

In [307]:
# read in model predictions
c_preds = pd.read_csv(pred_dir + 'C_Predictions_LGBM_'+str(curr_date)+'.csv')
w_preds = pd.read_csv(pred_dir + 'W_Predictions_LGBM_'+str(curr_date)+'.csv')
d_preds = pd.read_csv(pred_dir + 'D_Predictions_LGBM_'+str(curr_date)+'.csv')
g_preds = pd.read_csv(pred_dir + 'G_Predictions_LGBM_'+str(curr_date)+'.csv')

# concat predictions to a single dataframe
all_preds = pd.concat([c_preds, w_preds, d_preds, g_preds]).reset_index(drop = True)
all_preds = all_preds.drop(['Unnamed: 0'], axis = 1)

In [308]:
all_preds = all_preds.rename(columns = {'Name' : 'Nickname'})

# fix names with accent marks/suffixes
import unidecode

for i in range(len(all_preds)):
    # accent mark fix
    all_preds.iloc[i, 2] = unidecode.unidecode(all_preds.iloc[i, 2])
    # drop Jrs.
    if all_preds.iloc[i, 2][-3:] == 'Jr.' or all_preds.iloc[i, 2][-2:] == 'IV':
        all_preds.iloc[i, 2] = all_preds.iloc[i, 2][:-3]

# read in draftkings salaries
dk_df = pd.read_csv(roster_file)

# rename columns to match FD columns
dk_df = dk_df.rename(columns = {'Name' : 'Nickname', 'AvgPointsPerGame' : 'FPPG'})

In [309]:
dk_df

,Position,Name + ID,Nickname,ID,Roster Position,Salary,Game Info,TeamAbbrev,FPPG
0,G,Frederik Andersen (26580653),Frederik Andersen,26580653,CPT,16200,LA@CAR 01/31/2023 07:00PM ET,CAR,12.29
1,G,Antti Raanta (26580694),Antti Raanta,26580694,CPT,15900,LA@CAR 01/31/2023 07:00PM ET,CAR,11.35
2,G,Pheonix Copley (26580654),Pheonix Copley,26580654,CPT,15300,LA@CAR 01/31/2023 07:00PM ET,LA,12.64
3,D,Brent Burns (26580655),Brent Burns,26580655,CPT,15000,LA@CAR 01/31/2023 07:00PM ET,CAR,11.43
4,G,Jonathan Quick (26580695),Jonathan Quick,26580695,CPT,15000,LA@CAR 01/31/2023 07:00PM ET,LA,8.03
...,...,...,...,...,...,...,...,...,...
117,C,Tyler Madden (26580648),Tyler Madden,26580648,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,LA,0.00
118,D,Brandt Clarke (26580649),Brandt Clarke,26580649,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,LA,5.30
119,D,Jordan Spence (26580650),Jordan Spence,26580650,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,LA,3.30
120,D,Maxime Lajoie (26580651),Maxime Lajoie,26580651,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,CAR,1.00


In [310]:
# merge predictions and DK df so salaries are available
all_preds_merged = pd.merge(all_preds, dk_df, on='Nickname')

In [311]:
# drop players w/ 0 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] != 0)].reset_index(drop = True)

# keep players w/ > 2.1 FPPG
all_preds_merged = all_preds_merged[(all_preds_merged['FPPG'] >= 2.1)].reset_index(drop = True)

# drop players based on salary cutoffs
# makes sure that bad players are excluded for each position
all_preds_merged = all_preds_merged[(all_preds_merged.Salary >= 6000)].reset_index(drop = True)

In [312]:
# filter_df for pulp
filter_df = all_preds_merged.copy()
filter_df['Pos_CPT'] = np.where(filter_df['Roster Position']=='CPT', 1, 0)
filter_df['Pos_FLEX'] = np.where(filter_df['Roster Position']=='FLEX', 1, 0)
filter_df['CPT_Rank'] = filter_df.loc[filter_df['Pos_CPT'] == 1]['Prediction_LGBM'].rank(method = 'min', ascending = False)
filter_df['CPT_Rank'] = filter_df['CPT_Rank'].fillna(1000)
filter_df['T3_CPT'] = np.where(filter_df['CPT_Rank'] <= 3, 1, 0)

# only keep top 3 ranked captains and all flex players
filter_df = filter_df.loc[(filter_df['T3_CPT'] == 1) | (filter_df['Pos_FLEX'] == 1)]
filter_df = filter_df.reset_index(drop=True)

# remaining cols. for filter_df
filter_df['MaxUsed'] = 0
filter_df['Pos_C'] = np.where(filter_df['Position']=='C', 1, 0)
filter_df['Pos_W'] = np.where(filter_df['Position']=='W', 1, 0)
filter_df['Pos_D'] = np.where(filter_df['Position']=='D', 1, 0)
filter_df['Pos_G'] = np.where(filter_df['Position']=='G', 1, 0)
filter_df['Pos_All'] = 1

In [313]:
# ensure players are from different teams (at least 1 player from each team)
teams = filter_df['Team'].value_counts().index.tolist()
filter_df['Team_1'] = np.where((filter_df['Team']==teams[0]), 1, 0)
filter_df['Team_2'] = np.where((filter_df['Team']==teams[1]), 1, 0)

In [314]:
encoded = pd.get_dummies(filter_df['Nickname']) #<-- One-Hot Encoding 

In [315]:
filter_df = filter_df.join(encoded) #<-- joining it to the raw_data table

In [316]:
encoded.columns.tolist()

['Adrian Kempe',
 'Alex Iafallo',
 'Andrei Svechnikov',
 'Antti Raanta',
 'Anze Kopitar',
 'Brady Skjei',
 'Brent Burns',
 'Brett Pesce',
 'Drew Doughty',
 'Frederik Andersen',
 'Jonathan Quick',
 'Kevin Fiala',
 'Martin Necas',
 'Max Pacioretty',
 'Pheonix Copley',
 'Phillip Danault',
 'Sean Durzi',
 'Sebastian Aho',
 'Teuvo Teravainen',
 'Trevor Moore',
 'Viktor Arvidsson']

In [317]:
import pulp

def pred_lineup_singlegame_DK(df, lineups, overlap, player_names = [0], nonplayer_names = [0]):
    #Initialize our PuLP problem, we choose pulp.LpMinimize because we are going to want the lowest ranked players numerically. (1st is better than 2nd, technically smaller though)
    prob = pulp.LpProblem('NHL_DK', pulp.LpMaximize)
    
    #Count How Many Different Players We Have (Including Team D)
    num_all = df.shape[0]
    
    #Create Salary Cap Variable
    salary_cap = 50000
    
    #Create List Of PuLP player variables
    players_lineup = [pulp.LpVariable("Player_{}".format(i+1), cat="Binary") for i in range(num_all)]
    
    #Tell our PuLP solver that we want our lineup to have our prediction field to be maximized
    prob += pulp.lpSum((pulp.lpSum(df.loc[i, 'Prediction_LGBM'] * players_lineup[i] for i in range(num_all))))
    
    #Make Sure To Select 6 Players Total
    prob += ((pulp.lpSum(df.loc[i, 'Pos_All'] * players_lineup[i] for i in range(num_all))) == 6)
    
    # Make sure only 1 CPT is selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_CPT'] * players_lineup[i] for i in range(num_all)))  == 1)
    
    # Make sure 5 FLEX players are selected
    prob += ((pulp.lpSum(df.loc[i, 'Pos_FLEX'] * players_lineup[i] for i in range(num_all)))  == 5)
    
    #add the salary constraint
    prob += ((pulp.lpSum(df.loc[i, 'Salary'] * players_lineup[i] for i in range(num_all))) <= salary_cap)
    
    #Add Our MaxUsed Constraint
    prob += ((pulp.lpSum(df.loc[i, 'MaxUsed'] * players_lineup[i] for i in range(num_all))) == 0)
    
    #Add constraints for teams so at least 1 player from each team is selected
    prob += ((pulp.lpSum(df.loc[i, 'Team_1'] * players_lineup[i] for i in range(num_all))) <= 5)
    prob += ((pulp.lpSum(df.loc[i, 'Team_2'] * players_lineup[i] for i in range(num_all))) <= 5)
    
    #select specific player, if indicated
    if player_names != [0]:
        for player_name in player_names:
            prob += ((pulp.lpSum(df.loc[i, player_name] * players_lineup[i] for i in range(num_all)))  == 1)
            
    #remove specific player from consideration, if indicated
    if nonplayer_names != [0]:
        for nonplayer_name in nonplayer_names:
            prob += ((pulp.lpSum(df.loc[i, nonplayer_name] * players_lineup[i] for i in range(num_all)))  == 0)
    
    # add constraint to select only 1 of each player
    for player in encoded.columns.tolist():
        prob += ((pulp.lpSum(df.loc[i, player] * players_lineup[i] for i in range(num_all))) <= 1)
    
    #variance constraints - each lineup can't have more than the num overlap of any combination of players in any previous lineups
    for i in range(len(lineups)):
        prob += ((pulp.lpSum(lineups[i][k] * players_lineup[k] for k in range(num_all))) <= overlap)
        
    #solve the problem
    status = prob.solve()
    
    #check if the optimizer found an optimal solution
    if status != pulp.LpStatusOptimal:
        print('Only {} feasible lineups produced'.format(len(lineups)), '\n')
        return None
    lineup_copy = []
    for i in range(num_all):
        if players_lineup[i].varValue >= 0.9 and players_lineup[i].varValue <= 1.1:
            lineup_copy.append(1)
        else:
            lineup_copy.append(0)

    return lineup_copy

In [318]:
# function to tranform lineups resulting from PULP optimization
def transform_lineup_DK(lineup, lineup_num):
    positions, names, ids, salaries = [], [], [], []
    game_infos, teams, ppgs, ranks = [], [], [], []
    predictions = []
    pos_C = []
    pos_W = []
    pos_D = []
    pos_G = []
    ros_pos = []

    players_lineup = lineup[:len(lineup)]
    for num, player in enumerate(players_lineup):
        if player == 1:
            positions.append(filter_df.loc[num, 'Position'])
            names.append(filter_df.loc[num, 'Nickname'])
            ids.append(filter_df.loc[num, 'ID'])
            salaries.append(filter_df.loc[num, 'Salary'])
            teams.append(filter_df.loc[num, 'Team'])
            ppgs.append(filter_df.loc[num, 'FPPG'])
            ranks.append(filter_df.loc[num, 'PredictRank_LGBM'])
            predictions.append(filter_df.loc[num, 'Prediction_LGBM'])
            pos_C.append(filter_df.loc[num, 'Pos_C']),
            pos_W.append(filter_df.loc[num, 'Pos_W']),
            pos_D.append(filter_df.loc[num, 'Pos_D']),
            pos_G.append(filter_df.loc[num, 'Pos_G']),
            ros_pos.append(filter_df.loc[num, 'Roster Position'])
            
    df = pd.DataFrame({'Position':positions,'Name':names, 'ID':ids, 'Salary':salaries, 
                       'TeamAbbrev':teams,'FPPG':ppgs,
                       'Rank':ranks,'LineupNum':lineup_num, 'Prediction':predictions,
                       'Pos_C':pos_C, 'Pos_W':pos_W, 'Pos_D':pos_D, 'Pos_G':pos_G,'Roster Position':ros_pos})
    return df

In [319]:
csv_dfs = []
# function to display transformed lineups optimized by PULP
def gen_lineups_singlegame_DK(df, num_lineups, overlap, player_names = [0], nonplayer_names = [0]):
    lineup_list = []
    for i in range(num_lineups):
        lineup_list.append(pred_lineup_singlegame_DK(df, lineup_list, overlap, player_names, nonplayer_names))
    print('Number of generated lineups:', len(lineup_list))
    
    for i in range(len(lineup_list)):
        lineup_df = transform_lineup_DK(lineup_list[i], i+1)
        
        # custom sort for lineup
        sort_dict = {'C': 0, 'W': 1, 'D': 2, 'G': 3} 
        display(lineup_df[['Name', 'Salary', 'FPPG', 'TeamAbbrev', 'Prediction', 'Roster Position']].sort_values(by=['Roster Position','Prediction'], ascending = [True, False]))
        
        print('salary sum:', lineup_df['Salary'].sum(), '\nlineup FPPG mean:', lineup_df['FPPG'].mean(), '\nlineup Rank mean:', lineup_df['Rank'].mean(), '\nlineup Predicted FP:', lineup_df['Prediction'].sum())
        v_counts = lineup_df['Position'].value_counts()
        values = lineup_df['Position'].value_counts().keys().tolist()
        counts = lineup_df['Position'].value_counts().tolist()

In [324]:
# lineup generation (don't enter player name as 4th arg. (which should be a list) unless specific player is desired)
gen_lineups_singlegame_DK(filter_df, 5, 3, [0],[0])

Number of generated lineups: 5


,Name,Salary,FPPG,TeamAbbrev,Prediction,Roster Position
2,Martin Necas,12900,12.12,CAR,14.959367,CPT
0,Sebastian Aho,9200,13.46,CAR,15.882780,FLEX
3,Andrei Svechnikov,8800,11.87,CAR,12.777016,FLEX
1,Phillip Danault,6400,8.93,LA,10.215633,FLEX
4,Teuvo Teravainen,6600,7.32,CAR,9.542190,FLEX
5,Brett Pesce,6000,7.74,CAR,8.672529,FLEX


salary sum: 49900 
lineup FPPG mean: 10.24 
lineup Rank mean: 8.0 
lineup Predicted FP: 72.0495132877051


,Name,Salary,FPPG,TeamAbbrev,Prediction,Roster Position
0,Sebastian Aho,13800,13.46,CAR,15.882780,CPT
2,Martin Necas,8600,12.12,CAR,14.959367,FLEX
3,Viktor Arvidsson,7600,9.76,LA,11.390873,FLEX
1,Phillip Danault,6400,8.93,LA,10.215633,FLEX
4,Alex Iafallo,7000,8.43,LA,9.741570,FLEX
5,Teuvo Teravainen,6600,7.32,CAR,9.542190,FLEX


salary sum: 50000 
lineup FPPG mean: 10.003333333333332 
lineup Rank mean: 9.333333333333334 
lineup Predicted FP: 71.73241231160671


...

salary sum: 49900 
lineup FPPG mean: 10.094999999999999 
lineup Rank mean: 10.666666666666666 
lineup Predicted FP: 70.54302742823597


,Name,Salary,FPPG,TeamAbbrev,Prediction,Roster Position
0,Martin Necas,12900,12.12,CAR,14.959367,CPT
1,Kevin Fiala,9600,12.06,LA,14.937634,FLEX
2,Viktor Arvidsson,7600,9.76,LA,11.390873,FLEX
3,Alex Iafallo,7000,8.43,LA,9.741570,FLEX
4,Teuvo Teravainen,6600,7.32,CAR,9.542190,FLEX
5,Brett Pesce,6000,7.74,CAR,8.672529,FLEX


salary sum: 49700 
lineup FPPG mean: 9.571666666666667 
lineup Rank mean: 10.0 
lineup Predicted FP: 69.24416225641573


,Name,Salary,FPPG,TeamAbbrev,Prediction,Roster Position
3,Kevin Fiala,14400,12.06,LA,14.937634,CPT
2,Martin Necas,8600,12.12,CAR,14.959367,FLEX
0,Anze Kopitar,8000,10.21,LA,10.527616,FLEX
1,Phillip Danault,6400,8.93,LA,10.215633,FLEX
4,Teuvo Teravainen,6600,7.32,CAR,9.542190,FLEX
5,Brett Pesce,6000,7.74,CAR,8.672529,FLEX


salary sum: 50000 
lineup FPPG mean: 9.73 
lineup Rank mean: 8.333333333333334 
lineup Predicted FP: 68.85496747688904


In [321]:
filter_df

,Season,Date,Nickname,Team,Opponent,Prediction_LGBM,PredictRank_LGBM,Position,Name + ID,ID,...,Kevin Fiala,Martin Necas,Max Pacioretty,Pheonix Copley,Phillip Danault,Sean Durzi,Sebastian Aho,Teuvo Teravainen,Trevor Moore,Viktor Arvidsson
0,2022,20230131,Sebastian Aho,CAR,LA,15.882780,1.0,C,Sebastian Aho (26580657),26580657,...,0,0,0,0,0,0,1,0,0,0
1,2022,20230131,Sebastian Aho,CAR,LA,15.882780,1.0,C,Sebastian Aho (26580596),26580596,...,0,0,0,0,0,0,1,0,0,0
2,2022,20230131,Anze Kopitar,LA,CAR,10.527616,7.0,C,Anze Kopitar (26580601),26580601,...,0,0,0,0,0,0,0,0,0,0
3,2022,20230131,Phillip Danault,LA,CAR,10.215633,9.0,C,Phillip Danault (26580607),26580607,...,0,0,0,0,1,0,0,0,0,0
4,2022,20230131,Adrian Kempe,LA,CAR,9.686122,10.0,RW,Adrian Kempe (26580599),26580599,...,0,0,0,0,0,0,0,0,0,0
5,2022,20230131,Trevor Moore,LA,CAR,3.671359,95.0,C,Trevor Moore (26580636),26580636,...,0,0,0,0,0,0,0,0,1,0
6,2022,20230131,Martin Necas,CAR,LA,14.959367,5.0,RW,Martin Necas (26580659),26580659,...,0,1,0,0,0,0,0,0,0,0
7,2022,20230131,Martin Necas,CAR,LA,14.959367,5.0,RW,Martin Necas (26580598),26580598,...,0,1,0,0,0,0,0,0,0,0
8,2022,20230131,Kevin Fiala,LA,CAR,14.937634,7.0,LW,Kevin Fiala (26580656),26580656,...,1,0,0,0,0,0,0,0,0,0
9,2022,20230131,Kevin Fiala,LA,CAR,14.937634,7.0,LW,Kevin Fiala (26580595),26580595,...,1,0,0,0,0,0,0,0,0,0


In [322]:
filter_df[['Nickname', 'Prediction_LGBM', 'Roster Position', 'Pos_CPT', 'Pos_FLEX', 'Pos_All']]

,Nickname,Prediction_LGBM,Roster Position,Pos_CPT,Pos_FLEX,Pos_All
0,Sebastian Aho,15.882780,CPT,1,0,1
1,Sebastian Aho,15.882780,FLEX,0,1,1
2,Anze Kopitar,10.527616,FLEX,0,1,1
3,Phillip Danault,10.215633,FLEX,0,1,1
4,Adrian Kempe,9.686122,FLEX,0,1,1
5,Trevor Moore,3.671359,FLEX,0,1,1
6,Martin Necas,14.959367,CPT,1,0,1
7,Martin Necas,14.959367,FLEX,0,1,1
8,Kevin Fiala,14.937634,CPT,1,0,1
9,Kevin Fiala,14.937634,FLEX,0,1,1


In [174]:
dk_df

,Position,Name + ID,Nickname,ID,Roster Position,Salary,Game Info,TeamAbbrev,FPPG
0,G,Frederik Andersen (26580653),Frederik Andersen,26580653,CPT,16200,LA@CAR 01/31/2023 07:00PM ET,CAR,12.29
1,G,Antti Raanta (26580694),Antti Raanta,26580694,CPT,15900,LA@CAR 01/31/2023 07:00PM ET,CAR,11.35
2,G,Pheonix Copley (26580654),Pheonix Copley,26580654,CPT,15300,LA@CAR 01/31/2023 07:00PM ET,LA,12.64
3,D,Brent Burns (26580655),Brent Burns,26580655,CPT,15000,LA@CAR 01/31/2023 07:00PM ET,CAR,11.43
4,G,Jonathan Quick (26580695),Jonathan Quick,26580695,CPT,15000,LA@CAR 01/31/2023 07:00PM ET,LA,8.03
...,...,...,...,...,...,...,...,...,...
117,C,Tyler Madden (26580648),Tyler Madden,26580648,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,LA,0.00
118,D,Brandt Clarke (26580649),Brandt Clarke,26580649,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,LA,5.30
119,D,Jordan Spence (26580650),Jordan Spence,26580650,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,LA,3.30
120,D,Maxime Lajoie (26580651),Maxime Lajoie,26580651,FLEX,2000,LA@CAR 01/31/2023 07:00PM ET,CAR,1.00
